# Analisis Simulacion

## Analisis General - Simulación Base

### Pre-procesamiento

In [1]:
import pandas as pd

# 1. Leer los archivos
arma_df = pd.read_excel("./datos/Simulacion/Base/resultados_140_ARMA_FINAL.xlsx")
arima_df = pd.read_excel("./datos/Simulacion/Base/resultados_140_ARIMA_FINAL.xlsx")
setar_df = pd.read_excel("./datos/Simulacion/Base/resultados_140_SETAR_FINAL.xlsx")

# 2. Asignar la columna ESCENARIO a cada dataframe
arma_df['ESCENARIO'] = "Lineal Estacionario"
arima_df['ESCENARIO'] = "Lineal No estacionario"
setar_df['ESCENARIO'] = "No lineal Estacionario"

# 3. Juntarlos uno bajo el otro (Concatenar)
df_total = pd.concat([arma_df, arima_df, setar_df], ignore_index=True)

# Seleccionar solo las columnas requeridas
columnas_deseadas = [
    "Paso", "Config", "Dist", "Var", "Block Bootstrapping", 
    "Sieve Bootstrap", "LSPM", "LSPMW", "AREPD", "MCPS", 
    "AV-MCPS", "DeepAR", "EnCQR-LSTM", "ESCENARIO"
]
df_total = df_total[columnas_deseadas]

# Definimos cuáles son las columnas que representan a los modelos predictivos
modelos = [
    "Block Bootstrapping", "Sieve Bootstrap", "LSPM", "LSPMW", 
    "AREPD", "MCPS", "AV-MCPS", "DeepAR", "EnCQR-LSTM"
]

# 4. Guardar el dataframe consolidado
df_total.to_excel("./datos/Simulacion/Base/dataframe_consolidado.xlsx", index=False)

# 5. Generar y mostrar las tablas (Media y Mediana)
metricas = {'MEDIA': 'mean', 'MEDIANA': 'median'}

for nombre_metrica, funcion in metricas.items():
    # Calculamos el valor general según la métrica (mean o median)
    if funcion == 'mean':
        resumen_general = df_total[modelos].mean()
        resumen_escenarios = df_total.groupby('ESCENARIO')[modelos].mean().T
    else:
        resumen_general = df_total[modelos].median()
        resumen_escenarios = df_total.groupby('ESCENARIO')[modelos].median().T

    # Construimos la tabla final para esta métrica
    tabla_resumen = pd.DataFrame(index=modelos)
    tabla_resumen['General'] = resumen_general
    tabla_resumen['ARMA'] = resumen_escenarios['Lineal Estacionario']
    tabla_resumen['ARIMA'] = resumen_escenarios['Lineal No estacionario']
    tabla_resumen['SETAR'] = resumen_escenarios['No lineal Estacionario']

    # Determinar el Mejor_Escenario (valor mínimo entre los tres escenarios)
    escenarios_cols = ['ARMA', 'ARIMA', 'SETAR']
    tabla_resumen['Mejor_Escenario'] = tabla_resumen[escenarios_cols].idxmin(axis=1)

    # Imprimir resultado
    print(f"\n--- Tabla Comparativa de Modelos (Basada en {nombre_metrica}) ---")
    print(tabla_resumen.reset_index().rename(columns={'index': 'Modelo'}).to_string(index=False))


--- Tabla Comparativa de Modelos (Basada en MEDIA) ---
             Modelo  General     ARMA     ARIMA    SETAR Mejor_Escenario
Block Bootstrapping 4.275512 0.947365 11.251901 0.627270           SETAR
    Sieve Bootstrap 0.570491 0.547725  0.547481 0.616268           ARIMA
               LSPM 0.845497 0.811287  1.064804 0.660398           SETAR
              LSPMW 1.572319 0.960743  3.079645 0.676570           SETAR
              AREPD 3.880255 0.936671 10.031183 0.672910           SETAR
               MCPS 1.562693 0.759453  3.231805 0.696822           SETAR
            AV-MCPS 1.588747 0.740911  3.341841 0.683490           SETAR
             DeepAR 1.836024 0.568693  4.329124 0.610255            ARMA
         EnCQR-LSTM 2.454851 0.843920  5.850490 0.670144           SETAR

--- Tabla Comparativa de Modelos (Basada en MEDIANA) ---
             Modelo  General     ARMA    ARIMA    SETAR Mejor_Escenario
Block Bootstrapping 0.989417 0.653853 5.275960 0.540499           SETAR
    Sieve Bo

### Analisis general

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import normaltest
import warnings
from pathlib import Path
import itertools

warnings.filterwarnings('ignore')

# Configuración general
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'
sns.set_palette("husl")

# Crear carpeta de resultados
output_dir = Path("./Resultados_analisis/Simulacion_base")
output_dir.mkdir(parents=True, exist_ok=True)

# Crear carpeta de interacciones
interactions_dir = output_dir / "Interacciones"
interactions_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_excel("./datos/Simulacion/Base/dataframe_consolidado.xlsx")

# 1) CAMBIO DE NOMBRES DE ESCENARIOS
df['ESCENARIO'] = df['ESCENARIO'].replace({
    "Lineal Estacionario": "Lineal Estacionario (ARMA)",
    "Lineal No estacionario": "Lineal No Estacionario (ARIMA)",
    "No lineal Estacionario": "No lineal Estacionario (SETAR)"
})

# Identificar columnas de modelos
var_cols = ['Paso', 'Config', 'Dist', 'Var', 'ESCENARIO']
original_model_cols = [col for col in df.columns if col not in var_cols]

# 2) ORGANIZAR MODELOS POR RENDIMIENTO EN "Lineal No Estacionario (ARIMA)" (Menor a mayor)
target_scenario = "Lineal No Estacionario (ARIMA)"
model_order_scores = df[df['ESCENARIO'] == target_scenario][original_model_cols].mean().sort_values()
model_cols = list(model_order_scores.index)

print("Nuevo orden de modelos (basado en Lineal No Estacionario (ARIMA)):")
for i, m in enumerate(model_cols, 1):
    print(f"{i}. {m}: {model_order_scores[m]:.4f}")

# Mapeo de escenarios
escenarios_map = {
    'Lineal Estacionario (ARMA)': 'Lineal Estacionario (ARMA)',
    'Lineal No Estacionario (ARIMA)': 'Lineal No Estacionario (ARIMA)',
    'No lineal Estacionario (SETAR)': 'No lineal Estacionario (SETAR)'
}

# Definir colores para cada modelo
palette = sns.color_palette("husl", len(model_cols))
model_colors = {model: palette[i] for i, model in enumerate(model_cols)}

# ====================================================================================
# SECCIÓN 1: RENDIMIENTO POR ESCENARIOS
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 1: RENDIMIENTO POR ESCENARIOS")
print("="*80)

def plot_performance_by_scenario():
    fig, ax = plt.subplots(figsize=(14, 8))
    
    scenarios = ["Lineal Estacionario (ARMA)", "Lineal No Estacionario (ARIMA)", "No lineal Estacionario (SETAR)"]
    x = np.arange(len(model_cols))
    width = 0.25 
    
    scenario_colors = {
        'Lineal Estacionario (ARMA)': '#5D3FD3',    
        'Lineal No Estacionario (ARIMA)': '#808080', 
        'No lineal Estacionario (SETAR)': '#00A36C'  
    }
    
    for idx, scenario in enumerate(scenarios):
        means = [df[df['ESCENARIO'] == scenario][model].mean() for model in model_cols]
        position = x + (idx - 1) * width
        
        bars = ax.bar(position, means, width, label=scenario, 
                     color=scenario_colors[scenario], alpha=0.85, edgecolor='black', linewidth=0.5)
        
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2f}', ha='center', va='bottom', fontsize=7, rotation=0)
    
    ax.set_xlabel('Modelo (Ordenados por desempeño en ARIMA)', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title('Rendimiento de Modelos por Escenario (ECRPS)', fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(model_cols, rotation=45, ha='right', fontsize=9)
    ax.legend(loc='upper left', ncol=1, fontsize=10, framealpha=0.9)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '1.1_rendimiento_por_escenario.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 1.1 guardada")

plot_performance_by_scenario()

def plot_relative_performance():
    base_scenario = 'Lineal Estacionario (ARMA)'
    scenarios_compare = ['Lineal No Estacionario (ARIMA)', 'No lineal Estacionario (SETAR)']
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    y = np.arange(len(model_cols))
    height = 0.35  
    
    for idx, scenario in enumerate(scenarios_compare):
        changes = []
        for model in model_cols:
            base_value = df[df['ESCENARIO'] == base_scenario][model].mean()
            scenario_value = df[df['ESCENARIO'] == scenario][model].mean()
            pct_change = ((scenario_value - base_value) / base_value) * 100
            changes.append(pct_change)
        
        position = y + idx * height
        bars = ax.barh(position, changes, height, label=scenario, alpha=0.85, edgecolor='black', linewidth=0.5)
        
        for bar, val in zip(bars, changes):
            width = bar.get_width()
            ax.text(width + (1 if width > 0 else -1), bar.get_y() + bar.get_height()/2.,
                   f'{val:+.1f}%', ha='left' if val > 0 else 'right', 
                   va='center', fontsize=7)
    
    ax.set_yticks(y + height / 2)
    ax.set_yticklabels(model_cols, fontsize=10)
    ax.set_xlabel('Cambio Relativo (%)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title(f'Cambio Relativo en ECRPS vs. {base_scenario}', fontsize=14, fontweight='bold', pad=20)
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1.5)
    ax.legend(loc='best', fontsize=10, framealpha=0.9)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '1.2_cambio_relativo_escenario_base.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 1.2 guardada")

plot_relative_performance()

# ====================================================================================
# SECCIÓN 2: ANÁLISIS POR CONFIG
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 2: ANÁLISIS POR CONFIG")
print("="*80)

def plot_zscore_heatmap_config(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Z-scores de ECRPS por Configuración ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Z-scores de ECRPS por Configuración (General)'
    
    pivot_data = data_filtered.groupby('Config')[model_cols].mean()
    z_scores = pivot_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(z_scores.T, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, cbar_kws={'label': 'Z-score'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=12, fontweight='bold', pad=20)
    ax.set_xlabel('Configuración', fontsize=11)
    ax.set_ylabel('Modelo', fontsize=11)
    
    plt.tight_layout()
    filename = f'2.1{suffix}_zscore_config.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 2.1{suffix} guardada")

plot_zscore_heatmap_config()
plot_zscore_heatmap_config('Lineal Estacionario (ARMA)', '.a')
plot_zscore_heatmap_config('Lineal No Estacionario (ARIMA)', '.b')
plot_zscore_heatmap_config('No lineal Estacionario (SETAR)', '.c')

def plot_variability_config(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Configuración ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Configuración (General)'
    
    pivot_std = data_filtered.groupby('Config')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    fmt = '.2f' if suffix == '' else '.4f'
    sns.heatmap(pivot_std.T, annot=True, fmt=fmt, cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Configuración', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'2.2{suffix}_variabilidad_config.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 2.2{suffix} guardada")

plot_variability_config()
plot_variability_config('Lineal Estacionario (ARMA)', '.a')
plot_variability_config('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_config('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 3: ANÁLISIS POR DIST
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 3: ANÁLISIS POR DIST")
print("="*80)

def plot_zscore_heatmap_dist(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Z-scores de ECRPS por Distribución ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Z-scores de ECRPS por Distribución (General)'
    
    pivot_data = data_filtered.groupby('Dist')[model_cols].mean()
    z_scores = pivot_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(z_scores.T, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, cbar_kws={'label': 'Z-score'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=12, fontweight='bold', pad=20)
    ax.set_xlabel('Distribución', fontsize=11)
    ax.set_ylabel('Modelo', fontsize=11)
    
    plt.tight_layout()
    filename = f'3.1{suffix}_zscore_dist.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 3.1{suffix} guardada")

plot_zscore_heatmap_dist()
plot_zscore_heatmap_dist('Lineal Estacionario (ARMA)', '.a')
plot_zscore_heatmap_dist('Lineal No Estacionario (ARIMA)', '.b')
plot_zscore_heatmap_dist('No lineal Estacionario (SETAR)', '.c')

def plot_variability_dist(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Distribución ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Distribución (General)'
    
    pivot_std = data_filtered.groupby('Dist')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Distribución', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'3.2{suffix}_variabilidad_dist.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 3.2{suffix} guardada")

plot_variability_dist()
plot_variability_dist('Lineal Estacionario (ARMA)', '.a')
plot_variability_dist('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_dist('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 4: ANÁLISIS POR VAR
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 4: ANÁLISIS POR VAR")
print("="*80)

def plot_evolution_var(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Evolución de ECRPS por Varianza ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Evolución de ECRPS por Varianza (General)'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    var_values = sorted(data_filtered['Var'].unique())
    
    for model in model_cols:
        means = []
        for var in var_values:
            mean_val = data_filtered[data_filtered['Var'] == var][model].mean()
            means.append(mean_val)
        
        ax.plot(var_values, means, marker='o', label=model, color=model_colors[model],
                linewidth=2.5, markersize=7, alpha=0.85)
    
    ax.set_xlabel('Varianza', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=9, ncol=2, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    filename = f'4.1{suffix}_evolucion_var.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 4.1{suffix} guardada")

plot_evolution_var()
plot_evolution_var('Lineal Estacionario (ARMA)', '.a')
plot_evolution_var('Lineal No Estacionario (ARIMA)', '.b')
plot_evolution_var('No lineal Estacionario (SETAR)', '.c')

def plot_variability_var(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Varianza ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Varianza (General)'
    
    pivot_std = data_filtered.groupby('Var')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Varianza', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'4.2{suffix}_variabilidad_var.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 4.2{suffix} guardada")

plot_variability_var()
plot_variability_var('Lineal Estacionario (ARMA)', '.a')
plot_variability_var('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_var('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)")
print("="*80)

def plot_evolution_paso(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Evolución de ECRPS por Horizonte de Pronóstico ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Evolución de ECRPS por Horizonte de Pronóstico (General)'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    pasos = sorted(data_filtered['Paso'].unique())
    
    for model in model_cols:
        means = []
        for paso in pasos:
            mean_val = data_filtered[data_filtered['Paso'] == paso][model].mean()
            means.append(mean_val)
        
        ax.plot(pasos, means, marker='o', label=model, color=model_colors[model],
                linewidth=2.5, markersize=7, alpha=0.85)
    
    ax.set_xlabel('Horizonte de Pronóstico', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=9, ncol=2, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    filename = f'5.1{suffix}_evolucion_paso.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 5.1{suffix} guardada")

plot_evolution_paso()
plot_evolution_paso('Lineal Estacionario (ARMA)', '.a')
plot_evolution_paso('Lineal No Estacionario (ARIMA)', '.b')
plot_evolution_paso('No lineal Estacionario (SETAR)', '.c')

def plot_variability_paso(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Horizonte ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Horizonte (General)'
    
    pivot_std = data_filtered.groupby('Paso')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Horizonte de Pronóstico', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'5.2{suffix}_variabilidad_paso.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 5.2{suffix} guardada")

plot_variability_paso()
plot_variability_paso('Lineal Estacionario (ARMA)', '.a')
plot_variability_paso('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_paso('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 5.5: ANÁLISIS DE INTERACCIONES
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5.5: ANÁLISIS DE INTERACCIONES")
print("="*80)

# Función auxiliar para configurar el grid de modelos
def get_model_grid_axes(n_models):
    n_cols = 3
    n_rows = (n_models + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
    return fig, axes.flatten(), n_rows, n_cols

# INTERACCIÓN 3 (Lista 8): DIST × PASO - Subplots por Modelo
def plot_interaction_dist_paso(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Dist × Paso por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, n_rows, n_cols = get_model_grid_axes(len(model_cols))
    dists = sorted(data_filtered['Dist'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_dist = sns.color_palette("viridis", len(dists))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for d_idx, dist in enumerate(dists):
            means = [data_filtered[(data_filtered['Dist'] == dist) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='o', label=dist, color=colors_dist[d_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Distribución", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.7{suffix}_interaccion_dist_paso_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 8 (5.7) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 4 (Lista 9): DIST × VAR - Subplots por Modelo
def plot_interaction_dist_var(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Dist × Var por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    dists = sorted(data_filtered['Dist'].unique())
    vars_val = sorted(data_filtered['Var'].unique())
    colors_dist = sns.color_palette("magma", len(dists))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for d_idx, dist in enumerate(dists):
            means = [data_filtered[(data_filtered['Dist'] == dist) & (data_filtered['Var'] == v)][model].mean() for v in vars_val]
            ax.plot(vars_val, means, marker='s', label=dist, color=colors_dist[d_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Varianza')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Distribución", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.8{suffix}_interaccion_dist_var_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 9 (5.8) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 5 (Lista 10): CONFIG × PASO - Subplots por Modelo
def plot_interaction_config_paso(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Config × Paso por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    configs = sorted(data_filtered['Config'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_conf = sns.color_palette("tab10", len(configs))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for c_idx, config in enumerate(configs):
            means = [data_filtered[(data_filtered['Config'] == config) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='^', label=config, color=colors_conf[c_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Config", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.9{suffix}_interaccion_config_paso_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 10 (5.9) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 6 (Lista 11): VAR × HORIZONTE - Subplots por Modelo
def plot_interaction_var_horizonte(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Var × Horizonte por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    vars_val = sorted(data_filtered['Var'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_var = sns.color_palette("rocket", len(vars_val))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for v_idx, var in enumerate(vars_val):
            means = [data_filtered[(data_filtered['Var'] == var) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='d', label=f'Var {var}', color=colors_var[v_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Varianza", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.10{suffix}_interaccion_var_horizonte_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 11 (5.10) {suffix} guardada: Subplots por Modelo")


# ====================================================================================
# EJECUCIÓN DE LAS NUEVAS FUNCIONES
# ====================================================================================

for sc_name, sc_suf in [ (None, ''), ('Lineal Estacionario (ARMA)', '.a'), 
                        ('Lineal No Estacionario (ARIMA)', '.b'), 
                        ('No lineal Estacionario (SETAR)', '.c') ]:
    plot_interaction_dist_paso(sc_name, sc_suf)
    plot_interaction_dist_var(sc_name, sc_suf)
    plot_interaction_config_paso(sc_name, sc_suf)
    plot_interaction_var_horizonte(sc_name, sc_suf)

# ====================================================================================
# SECCIÓN 6: ROBUSTEZ Y TEST DIEBOLD-MARIANO
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 6: ROBUSTEZ Y TEST DIEBOLD-MARIANO")
print("="*80)

def plot_robustness():
    fig, ax = plt.subplots(figsize=(12, 8))
    
    cv_data = []
    for model in model_cols:
        cv = df[model].std() / df[model].mean()
        cv_data.append((model, cv))
    
    cv_df = pd.DataFrame(cv_data, columns=['Modelo', 'CV'])
    
    # CAMBIO 1: Ordenar de menor a mayor CV para coherencia
    cv_df = cv_df.sort_values('CV')
    
    colors_cv = ['#2ecc71' if cv < cv_df['CV'].median() else '#e74c3c' 
                 for cv in cv_df['CV']]
    
    bars = ax.barh(cv_df['Modelo'], cv_df['CV'], color=colors_cv, alpha=0.8, edgecolor='black')
    
    for bar, cv in zip(bars, cv_df['CV']):
        width = bar.get_width()
        ax.text(width + 0.001, bar.get_y() + bar.get_height()/2.,
               f'{cv:.4f}', ha='left', va='center', fontsize=9)
    
    ax.set_xlabel('Coeficiente de Variación', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title('Robustez: Coeficiente de Variación\n(Ordenado de menor a mayor - Menor valor indica mayor estabilidad)', 
                  fontsize=14, fontweight='bold', pad=20)
    ax.axvline(x=cv_df['CV'].median(), color='black', linestyle='--', linewidth=1.5, alpha=0.5, label='Mediana')
    ax.legend(loc='best', fontsize=10)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '6.1_robustez_coeficiente_variacion.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 6.1 guardada (ordenada de menor a mayor CV)")

plot_robustness()

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano con fixed-smoothing asymptotics (Coroneo & Iacone, 2020)
    
    Usa Fixed-m asymptotics (kernel Daniell) que es más robusto en muestras pequeñas
    y mantiene buen desempeño en muestras grandes.
    
    Parameters:
    -----------
    errors1, errors2 : array-like
        Errores de pronóstico (ECRPS) de los dos modelos
    h : int
        Horizonte de pronóstico (forecast horizon)
    
    Returns:
    --------
    hln_dm_stat : float
        Estadístico con fixed-m asymptotics
    p_value : float
        P-valor usando distribución t-Student con 2m grados de libertad
    dm_stat : float
        Estadístico DM original (para referencia)
    """
    # Calcular diferencial de pérdida
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    if T < 2:
        return np.nan, np.nan, np.nan
    
    # Desviaciones de la media
    u = d - d_bar
    
    # Fixed-m: bandwidth recomendado en el paper
    m = max(1, int(np.floor(T**(1/3))))
    
    # Calcular periodograma
    from scipy.fft import fft
    
    # FFT de las desviaciones
    fft_u = fft(u)
    periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
    
    # WPE con kernel de Daniell: promedio de primeros m periodogramas
    # (excluyendo frecuencia 0)
    if m >= len(periodogram) - 1:
        m = len(periodogram) - 2
    
    sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
    
    if sigma_hat_sq <= 0:
        # Fallback: usar varianza simple
        sigma_hat_sq = np.var(d, ddof=1) / T
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
    
    # Estadístico DM
    dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
    
    # Fixed-m asymptotics: límite es t-Student con 2m grados de libertad
    df = 2 * m
    hln_dm_stat = dm_stat
    
    # P-valor usando t-Student
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat

def plot_dm_test_heatmap(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Test Diebold-Mariano Modificado (HLN-DM)\ncon Corrección de Bonferroni ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Test Diebold-Mariano Modificado (HLN-DM)\ncon Corrección de Bonferroni (General)'
    
    # Determinar el horizonte de pronóstico promedio
    h_forecast = int(data_filtered['Paso'].mean())
    
    n_models = len(model_cols)
    n_comparisons = n_models * (n_models - 1) / 2
    alpha = 0.05
    bonferroni_alpha = alpha / n_comparisons
    
    results_matrix = np.zeros((n_models, n_models))
    p_values = np.zeros((n_models, n_models))
    dm_stats = np.zeros((n_models, n_models))
    
    for i, model1 in enumerate(model_cols):
        for j, model2 in enumerate(model_cols):
            if i == j:
                results_matrix[i, j] = 0  
                p_values[i, j] = 1.0
                dm_stats[i, j] = 0
            elif i < j:
                errors1 = data_filtered[model1].values
                errors2 = data_filtered[model2].values
                hln_dm_stat, p_val, dm_original = modified_diebold_mariano_test(
                    errors1, errors2, h=h_forecast
                )
                
                p_values[i, j] = p_val
                p_values[j, i] = p_val
                dm_stats[i, j] = hln_dm_stat
                dm_stats[j, i] = -hln_dm_stat
                
                if p_val < bonferroni_alpha:
                    mean1 = np.mean(errors1)
                    mean2 = np.mean(errors2)
                    if mean1 < mean2:  
                        results_matrix[i, j] = 1  
                        results_matrix[j, i] = -1  
                    else:
                        results_matrix[i, j] = -1
                        results_matrix[j, i] = 1
                else:
                    results_matrix[i, j] = 0
                    results_matrix[j, i] = 0
    
    fig, ax = plt.subplots(figsize=(14, 11))
    cmap = plt.cm.colors.ListedColormap(['#e74c3c', '#fff9c4', '#2ecc71'])
    bounds = [-1.5, -0.5, 0.5, 1.5]
    norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)
    
    im = ax.imshow(results_matrix, cmap=cmap, norm=norm, aspect='auto')
    ax.set_xticks(np.arange(n_models))
    ax.set_yticks(np.arange(n_models))
    ax.set_xticklabels(model_cols, rotation=45, ha='right', fontsize=9)
    ax.set_yticklabels(model_cols, fontsize=9)
    
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                text = '-'
                color = 'black'
            else:
                val = results_matrix[i, j]
                p_val = p_values[i, j]
                dm_val = dm_stats[i, j]
                if val == 1:
                    text = f'✓\nHLN-DM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'white'
                elif val == -1:
                    text = f'✗\nHLN-DM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'white'
                else:
                    text = f'≈\nHLN-DM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'black'
            ax.text(j, i, text, ha='center', va='center', 
                   color=color, fontsize=6, fontweight='bold')
    
    # Obtener T para el título
    T = len(data_filtered)
    df_test = T - 1
    
    ax.set_title(title + f'\n(h={h_forecast}, T={T}, df={df_test}, α ajustado={bonferroni_alpha:.5f})', 
                 fontsize=11, fontweight='bold', pad=20)
    ax.set_xlabel('Modelo (Columna)', fontsize=11)
    ax.set_ylabel('Modelo (Fila)', fontsize=11)
    
    legend_elements = [
        plt.Rectangle((0,0),1,1, facecolor='#2ecc71', label='Fila supera columna (p < α)'),
        plt.Rectangle((0,0),1,1, facecolor='#e74c3c', label='Columna supera fila (p < α)'),
        plt.Rectangle((0,0),1,1, facecolor='#fff9c4', label='Sin diferencia significativa (p ≥ α)')
    ]
    ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=9)
    
    # Agregar nota metodológica
    note_text = ('Nota: Se utiliza el test HLN-DM con corrección para muestras finitas.\n'
                 'Distribución: t-Student. Ajuste: Bonferroni para comparaciones múltiples.')
    plt.figtext(0.5, -0.02, note_text, ha='center', fontsize=8, style='italic', 
                wrap=True, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.tight_layout()
    filename = f'6.2{suffix}_dm_test_hln_bonferroni.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    
    # Guardar resultados en Excel
    results_df = pd.DataFrame(results_matrix, index=model_cols, columns=model_cols)
    pvalues_df = pd.DataFrame(p_values, index=model_cols, columns=model_cols)
    dmstats_df = pd.DataFrame(dm_stats, index=model_cols, columns=model_cols)
    
    summary_data = []
    for model in model_cols:
        idx = model_cols.index(model)
        victorias = int(np.sum(results_matrix[idx, :] == 1))
        derrotas = int(np.sum(results_matrix[idx, :] == -1))
        empates = int(np.sum(results_matrix[idx, :] == 0)) - 1
        mean_ecrps = data_filtered[model].mean()
        std_ecrps = data_filtered[model].std()
        cv_ecrps = std_ecrps / mean_ecrps
        summary_data.append({
            'Modelo': model,
            'Victorias': victorias,
            'Derrotas': derrotas,
            'Empates': empates,
            'Tasa_Victoria': f"{(victorias / (n_models - 1)) * 100:.1f}%",
            'ECRPS_Promedio': f"{mean_ecrps:.4f}",
            'ECRPS_Desv_Std': f"{std_ecrps:.4f}",
            'Coef_Variacion': f"{cv_ecrps:.4f}"
        })
    summary_df = pd.DataFrame(summary_data)
    
    # Información metodológica
    method_info = pd.DataFrame({
        'Parámetro': ['Horizonte de pronóstico (h)', 'Tamaño muestra (T)', 
                      'Grados libertad (df)', 'Alpha nominal', 
                      'Alpha ajustado (Bonferroni)', 'Número comparaciones',
                      'Distribución', 'Corrección aplicada'],
        'Valor': [h_forecast, T, df_test, alpha, bonferroni_alpha, 
                  int(n_comparisons), 't-Student', 'Harvey-Leybourne-Newbold (1997)']
    })
    
    with pd.ExcelWriter(output_dir / f'6.2{suffix}_dm_test_hln_resultados.xlsx') as writer:
        method_info.to_excel(writer, sheet_name='Metodologia', index=False)
        results_df.to_excel(writer, sheet_name='Matriz_Resultados')
        pvalues_df.to_excel(writer, sheet_name='P_valores')
        dmstats_df.to_excel(writer, sheet_name='Estadisticos_HLN_DM')
        summary_df.to_excel(writer, sheet_name='Resumen_Modelos', index=False)
    
    print(f"✓ Gráfica 6.2{suffix} guardada (Test HLN-DM con h={h_forecast}, T={T}, df={df_test})")

plot_dm_test_heatmap()
plot_dm_test_heatmap('Lineal Estacionario (ARMA)', '.a')
plot_dm_test_heatmap('Lineal No Estacionario (ARIMA)', '.b')
plot_dm_test_heatmap('No lineal Estacionario (SETAR)', '.c')

print("\n" + "="*80)
print("PROCESO COMPLETADO")
print("="*80)
print("\nMejoras implementadas:")
print("1. ✓ Orden consistente en gráficas 1.1 y 1.2")
print("2. ✓ Formato de 2 decimales en heatmap 2.2 general")
print("3. ✓ Solo Coeficiente de Variación en gráfica 6.1 (ordenado menor a mayor)")
print("4. ✓ Test Diebold-Mariano con corrección HLN y distribución t-Student")
print("5. ✓ Horizonte de pronóstico (h) incorporado en el análisis")
print("\n" + "="*80)
print("ANÁLISIS DE INTERACCIONES (Carpeta: Interacciones/)")
print("="*80)
print("6. ✓ Config × Var: Heatmaps por modelo (matriz 3×3)")
print("7. ✓ Config × Dist: Heatmaps por modelo (matriz 3×3)")
print("8. ✓ Dist × Paso: Gráficas de líneas por distribución")
print("9. ✓ Dist × Var: Gráficas de líneas por distribución")
print("10. ✓ Config × Paso: Gráficas de líneas por configuración")
print("11. ✓ Var × Horizonte: Gráficas de líneas por varianza")
print("="*80)
print(f"\nTotal de gráficas generadas en Interacciones/: {6 * 4} archivos")
print("(6 tipos de interacción × 4 escenarios: General + 3 específicos)")
print("="*80)

Nuevo orden de modelos (basado en Lineal No Estacionario (ARIMA)):
1. Sieve Bootstrap: 0.5475
2. LSPMW: 1.0636
3. LSPM: 1.0648
4. MCPS: 3.2318
5. AV-MCPS: 3.3418
6. DeepAR: 4.3291
7. EnCQR-LSTM: 5.8505
8. AREPD: 10.0312
9. Block Bootstrapping: 11.2519

SECCIÓN 1: RENDIMIENTO POR ESCENARIOS
✓ Gráfica 1.1 guardada
✓ Gráfica 1.2 guardada

SECCIÓN 2: ANÁLISIS POR CONFIG
✓ Gráfica 2.1 guardada
✓ Gráfica 2.1.a guardada
✓ Gráfica 2.1.b guardada
✓ Gráfica 2.1.c guardada
✓ Gráfica 2.2 guardada
✓ Gráfica 2.2.a guardada
✓ Gráfica 2.2.b guardada
✓ Gráfica 2.2.c guardada

SECCIÓN 3: ANÁLISIS POR DIST
✓ Gráfica 3.1 guardada
✓ Gráfica 3.1.a guardada
✓ Gráfica 3.1.b guardada
✓ Gráfica 3.1.c guardada
✓ Gráfica 3.2 guardada
✓ Gráfica 3.2.a guardada
✓ Gráfica 3.2.b guardada
✓ Gráfica 3.2.c guardada

SECCIÓN 4: ANÁLISIS POR VAR
✓ Gráfica 4.1 guardada
✓ Gráfica 4.1.a guardada
✓ Gráfica 4.1.b guardada
✓ Gráfica 4.1.c guardada
✓ Gráfica 4.2 guardada
✓ Gráfica 4.2.a guardada
✓ Gráfica 4.2.b guardada
✓ Gráfica

## Analisis Diferenciado

### Pre-procesamiento

In [23]:
import pandas as pd
import numpy as np

# 1. Cargar los archivos
path_diff = "./datos/Simulacion/Diferenciado/resultados_140_ARIMA_CON_DIFERENCIACION.xlsx"
path_no_diff = "./datos/Simulacion/Diferenciado/resultados_140_ARIMA_FINAL.xlsx"

arima_Diff_df = pd.read_excel(path_diff)
arima_df = pd.read_excel(path_no_diff)

# 2. Agregar la columna 'Diferenciacion' al dataframe que no la tiene
arima_df['Diferenciacion'] = 'No'

# 3. Unir los dataframes (uno debajo del otro)
# El orden de las columnas se ajustará automáticamente
df_unido = pd.concat([arima_Diff_df, arima_df], ignore_index=True)

# 4. Guardar la unión en la misma carpeta
path_salida = "./datos/Simulacion/Diferenciado/resultados_UNION_ARIMA.xlsx"
df_unido.to_excel(path_salida, index=False)

# 5. Crear la tabla comparativa (Resumen)
# Definimos las métricas/modelos a comparar
modelos_metricas = [
    'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 
    'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap'
]

# Calculamos el promedio para cada caso
resumen_no_diff = arima_df[modelos_metricas].mean()
resumen_diff = arima_Diff_df[modelos_metricas].mean()

# Construir el DataFrame comparativo
tabla_comparativa = pd.DataFrame({
    'ARIMA': resumen_no_diff,
    'ARIMA_Diff': resumen_diff
})

# Determinar el mejor escenario (el que tenga el valor menor, asumiendo que son errores)
tabla_comparativa['Mejor_Escenario'] = np.where(
    tabla_comparativa['ARIMA'] < tabla_comparativa['ARIMA_Diff'], 
    'ARIMA', 
    'ARIMA_Diff'
)

# Formatear la tabla para que se vea limpia
tabla_comparativa.index.name = 'Modelo'
tabla_comparativa = tabla_comparativa.sort_index()

# Mostrar resultados
print("Tabla Comparativa de Desempeño (Promedios):")
print(tabla_comparativa.to_string())

print(f"\nArchivo de unión guardado en: {path_salida}")

Tabla Comparativa de Desempeño (Promedios):
                         ARIMA  ARIMA_Diff Mejor_Escenario
Modelo                                                    
AREPD                10.031183    0.704149      ARIMA_Diff
AV-MCPS               3.324007    0.654257      ARIMA_Diff
Block Bootstrapping  11.251601    0.666133      ARIMA_Diff
DeepAR                4.329124    0.561822      ARIMA_Diff
EnCQR-LSTM            6.112344    0.880288      ARIMA_Diff
LSPM                  1.064804    0.648039      ARIMA_Diff
LSPMW                 3.079645    0.767172      ARIMA_Diff
MCPS                  3.218168    0.677471      ARIMA_Diff
Sieve Bootstrap       0.547481    0.546020      ARIMA_Diff

Archivo de unión guardado en: ./datos/Simulacion/Diferenciado/resultados_UNION_ARIMA.xlsx


### Analisis general

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.fft import fft
from pathlib import Path
import warnings

# Configuración inicial
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'

# ====================================================================================
# TEST DIEBOLD-MARIANO CON FIXED-M ASYMPTOTICS
# ====================================================================================
def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano con fixed-smoothing asymptotics (Coroneo & Iacone, 2020)
    
    Usa Fixed-m asymptotics (kernel Daniell) que es más robusto en muestras pequeñas
    y mantiene buen desempeño en muestras grandes.
    
    Parameters:
    -----------
    errors1, errors2 : array-like
        Errores de pronóstico (ECRPS) de los dos modelos
    h : int
        Horizonte de pronóstico (forecast horizon)
    
    Returns:
    --------
    hln_dm_stat : float
        Estadístico con fixed-m asymptotics
    p_value : float
        P-valor usando distribución t-Student con 2m grados de libertad
    dm_stat : float
        Estadístico DM original (para referencia)
    """
    # Calcular diferencial de pérdida
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    if T < 2:
        return np.nan, np.nan, np.nan
    
    # Desviaciones de la media
    u = d - d_bar
    
    # Fixed-m: bandwidth recomendado en el paper
    m = max(1, int(np.floor(T**(1/3))))
    
    # Calcular periodograma
    fft_u = fft(u)
    periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
    
    # WPE con kernel de Daniell: promedio de primeros m periodogramas
    # (excluyendo frecuencia 0)
    if m >= len(periodogram) - 1:
        m = len(periodogram) - 2
    
    sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
    
    if sigma_hat_sq <= 0:
        # Fallback: usar varianza simple
        sigma_hat_sq = np.var(d, ddof=1) / T
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
    
    # Estadístico DM
    dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
    
    # Fixed-m asymptotics: límite es t-Student con 2m grados de libertad
    df = 2 * m
    hln_dm_stat = dm_stat
    
    # P-valor usando t-Student
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat

# ====================================================================================
# 1. PREPARACIÓN DE DIRECTORIOS
# ====================================================================================
output_dir = Path("./Resultados_analisis/Multi_d")
output_dir.mkdir(parents=True, exist_ok=True)
plots_dir = output_dir / "Graficos_Analisis"
plots_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
path_excel = "./datos/Simulacion/Multi_D/resultados_ARIMA_d1_a_d10_DOBLE_MODALIDAD_COMPLETO.xlsx"
try:
    df = pd.read_excel(path_excel)
    print(f"✓ Datos cargados: {df.shape}")
    print(f"✓ Columnas: {df.columns.tolist()}")
except Exception as e:
    print(f"Error crítico: {e}")
    exit()

# Identificar columnas
var_cols = ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 'Distribución', 
            'Varianza', 'Modalidad', 'Valor_Observado']
model_cols = [col for col in df.columns if col not in var_cols]

print(f"✓ Modelos identificados: {model_cols}")
print(f"✓ Valores únicos de d: {sorted(df['d'].unique())}")

# ====================================================================================
# SECCIÓN 1: COMPARACIÓN INDIVIDUAL POR VALOR DE d
# ====================================================================================
def plot_individual_comparisons():
    print("\n=== Generando gráficos de barras con valores por d ===")
    
    # Para cada valor de d
    d_values = sorted(df['d'].unique())
    
    for d_val in d_values:
        stats_list = []
        subset_d = df[df['d'] == d_val]
        
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].mean()
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].mean()
            mejora = ((sin - con) / sin) * 100 if sin != 0 else 0
            stats_list.append({'Modelo': model, 'Sin': sin, 'Con': con, 'Mejora': mejora})
        
        df_stats = pd.DataFrame(stats_list).sort_values(by='Mejora', ascending=False)
        
        # 1.1 Barras Comparativas por d
        fig, ax = plt.subplots(figsize=(14, 6))
        x = np.arange(len(df_stats))
        width = 0.35
        
        b1 = ax.bar(x - width/2, df_stats['Sin'], width, label='Sin Dif.', 
                    color='#e74c3c', edgecolor='black', alpha=0.8)
        b2 = ax.bar(x + width/2, df_stats['Con'], width, label='Con Dif.', 
                    color='#2ecc71', edgecolor='black', alpha=0.8)
        
        ax.bar_label(b1, padding=3, fmt='%.3f', fontsize=7, rotation=45)
        ax.bar_label(b2, padding=3, fmt='%.3f', fontsize=7, rotation=45)
        
        ax.set_title(f'ECRPS Promedio por Modelo (d={d_val})', fontweight='bold', fontsize=12)
        ax.set_xticks(x)
        ax.set_xticklabels(df_stats['Modelo'], rotation=45, ha='right')
        ax.set_ylabel('ECRPS')
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.savefig(plots_dir / f'1.1_barras_ecrps_d{d_val}.png')
        plt.close()

# ====================================================================================
# SECCIÓN 2: HEATMAP d vs MODELO
# ====================================================================================
def plot_heatmap_d_vs_modelo():
    print("\n=== Generando Heatmap d vs Modelo ===")
    
    results = []
    d_values = sorted(df['d'].unique())
    
    for d_val in d_values:
        subset_d = df[df['d'] == d_val]
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].mean()
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].mean()
            mejora = ((sin - con) / sin) * 100 if sin != 0 else 0
            results.append({'d': d_val, 'Modelo': model, 'Mejora_%': mejora})
    
    df_mejora = pd.DataFrame(results)
    pivot = df_mejora.pivot(index='Modelo', columns='d', values='Mejora_%')
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(pivot, annot=True, fmt=".1f", cmap="RdYlGn", center=0, 
                cbar_kws={'label': 'Mejora (%)'}, linewidths=0.5)
    plt.title("Mejora Porcentual (%): Modelo vs Valor de d", fontweight='bold', fontsize=14)
    plt.xlabel("Valor de d (Grado de diferenciación)", fontweight='bold')
    plt.ylabel("Modelo", fontweight='bold')
    plt.tight_layout()
    plt.savefig(plots_dir / '2_heatmap_d_vs_modelo.png')
    plt.close()
    
    # Guardar tabla
    pivot.to_excel(output_dir / '2_tabla_d_vs_modelo.xlsx')

# ====================================================================================
# SECCIÓN 3: HEATMAP SIGNIFICANCIA CON DM TEST (d vs Diferenciación)
# ====================================================================================
def plot_heatmap_significancia():
    print("\n=== Generando Heatmap de Significancia d vs Diferenciación (DM Test) ===")
    
    results = []
    d_values = sorted(df['d'].unique())
    
    for d_val in d_values:
        subset_d = df[df['d'] == d_val]
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].values
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].values
            
            if len(sin) > 1 and len(con) > 1:
                # Usar DM test en lugar de t-test
                dm_stat, p_valor, _ = modified_diebold_mariano_test(sin, con, h=1)
                
                # Determinar dirección y significancia
                if p_valor < 0.05:
                    if sin.mean() > con.mean():
                        valor = 1  # Diferenciación es mejor
                    else:
                        valor = -1  # Sin diferenciación es mejor
                else:
                    valor = 0  # No significativo
            else:
                valor = 0
            
            results.append({'d': d_val, 'Modelo': model, 'Significancia': valor})
    
    df_sig = pd.DataFrame(results)
    pivot_sig = df_sig.pivot(index='Modelo', columns='d', values='Significancia')
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(pivot_sig, annot=True, fmt=".0f", cmap="RdYlGn", center=0,
                cbar_kws={'label': 'Significancia DM (-1: Sin Dif mejor | 0: No sig. | 1: Con Dif mejor)'},
                linewidths=0.5, vmin=-1, vmax=1)
    plt.title("Significancia Estadística (DM Test): d vs Diferenciación por Modelo", 
              fontweight='bold', fontsize=14)
    plt.xlabel("Valor de d", fontweight='bold')
    plt.ylabel("Modelo", fontweight='bold')
    plt.tight_layout()
    plt.savefig(plots_dir / '3_heatmap_significancia_d_DM.png')
    plt.close()
    
    pivot_sig.to_excel(output_dir / '3_tabla_significancia_d_DM.xlsx')

# ====================================================================================
# SECCIÓN 4: INTERACCIONES d vs Distribución, Config y Varianza
# ====================================================================================
def plot_interacciones_d():
    print("\n=== Generando Interacciones d vs Dist, Config y Varianza ===")
    
    # 4.1 d vs Distribución
    def get_improvement_d_group(group_col):
        results = []
        d_values = sorted(df['d'].unique())
        for d_val in d_values:
            subset_d = df[df['d'] == d_val]
            for val in subset_d[group_col].unique():
                subset = subset_d[subset_d[group_col] == val]
                for model in model_cols:
                    sin = subset[subset['Modalidad'] == 'SIN_DIFF'][model].mean()
                    con = subset[subset['Modalidad'] == 'CON_DIFF'][model].mean()
                    mejora = ((sin - con) / sin) * 100 if sin != 0 else 0
                    results.append({'d': d_val, group_col: val, 'Modelo': model, 'Mejora': mejora})
        return pd.DataFrame(results)
    
    # 4.1 Heatmap d vs Distribución (agrupado por modelo)
    df_dist = get_improvement_d_group('Distribución')
    
    for model in model_cols:
        subset = df_dist[df_dist['Modelo'] == model]
        pivot = subset.pivot(index='d', columns='Distribución', values='Mejora')
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(pivot, annot=True, fmt=".1f", cmap="RdYlGn", center=0)
        plt.title(f"Mejora (%) d vs Distribución - {model}", fontweight='bold')
        plt.xlabel("Distribución")
        plt.ylabel("Valor de d")
        plt.tight_layout()
        plt.savefig(plots_dir / f'4.1_heatmap_d_dist_{model}.png')
        plt.close()
    
    # 4.2 Heatmap d vs Varianza (líneas por modelo)
    df_var = get_improvement_d_group('Varianza')
    
    plt.figure(figsize=(14, 8))
    for model in model_cols:
        subset = df_var[df_var['Modelo'] == model]
        grouped = subset.groupby(['d', 'Varianza'])['Mejora'].mean().reset_index()
        for var_val in grouped['Varianza'].unique():
            data = grouped[grouped['Varianza'] == var_val]
            plt.plot(data['d'], data['Mejora'], marker='o', label=f'{model}-Var{var_val}', alpha=0.7)
    
    plt.axhline(0, color='black', linestyle='--', alpha=0.5)
    plt.title("Tendencia Mejora (%): d vs Varianza por Modelo", fontweight='bold')
    plt.xlabel("Valor de d")
    plt.ylabel("Mejora Porcentual")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(plots_dir / '4.2_linea_d_varianza.png')
    plt.close()
    
    # 4.3 Heatmap d vs Config (por modelo)
    if 'ARMA_base' in df.columns:
        df_config = get_improvement_d_group('ARMA_base')
        
        for model in model_cols:
            subset = df_config[df_config['Modelo'] == model]
            pivot = subset.pivot(index='d', columns='ARMA_base', values='Mejora')
            
            plt.figure(figsize=(12, 8))
            sns.heatmap(pivot, annot=True, fmt=".1f", cmap="RdYlGn", center=0)
            plt.title(f"Mejora (%) d vs Configuración Base - {model}", fontweight='bold')
            plt.xlabel("Configuración ARMA")
            plt.ylabel("Valor de d")
            plt.tight_layout()
            plt.savefig(plots_dir / f'4.3_heatmap_d_config_{model}.png')
            plt.close()

# ====================================================================================
# SECCIÓN 6: TABLA DM POR VALOR DE d (USANDO DM TEST)
# ====================================================================================
def run_dm_analysis():
    print("\n=== Calculando Tabla DM por valor de d (Fixed-m Asymptotics) ===")
    
    d_values = sorted(df['d'].unique())
    all_results = []
    
    for d_val in d_values:
        subset_d = df[df['d'] == d_val]
        
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].values
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].values
            
            if len(sin) > 1 and len(con) > 1:
                mean_sin, mean_con = sin.mean(), con.mean()
                mejora_pct = ((mean_sin - mean_con) / mean_sin) * 100 if mean_sin != 0 else 0
                
                # Usar DM test con fixed-m
                dm_stat, p_valor, dm_stat_original = modified_diebold_mariano_test(sin, con, h=1)
                significativo = 'Sí' if p_valor < 0.05 else 'No'
                
                if p_valor < 0.05:
                    conclusion = "Diferenciación mejora" if mejora_pct > 0 else "Sin dif mejor"
                else:
                    conclusion = "Sin diferencia significativa"
                
                all_results.append({
                    'd': d_val, 'Modelo': model, 
                    'ECRPS_Sin_Diff': mean_sin, 'ECRPS_Con_Diff': mean_con,
                    'Mejora_%': mejora_pct, 
                    'DM_Stat': dm_stat,
                    'p_valor': p_valor, 
                    'Significativo': significativo, 
                    'Conclusion': conclusion
                })
    
    dm_df = pd.DataFrame(all_results).sort_values(by=['d', 'Mejora_%'], ascending=[True, False])
    dm_df.to_excel(output_dir / '6_tabla_dm_por_d_fixed_m.xlsx', index=False)
    
    print(f"✓ Tabla DM guardada con {len(dm_df)} comparaciones")

# ====================================================================================
# SECCIÓN 9: EXCEL DETALLADO POR d (Agrupado, Ordenado)
# ====================================================================================
def generate_detailed_excel():
    print("\n=== Generando Excel detallado por d (ordenado por mejora) ===")
    file_path = output_dir / "9_Analisis_Detallado_Modelos_por_d.xlsx"
    
    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        for model in model_cols:
            # Agrupar por d, ARMA_base, Distribución, Varianza (promediando Pasos)
            temp = df.groupby(['d', 'ARMA_base', 'Distribución', 'Varianza', 'Modalidad'])[model].mean().reset_index()
            
            # Pivotar
            detailed = temp.pivot_table(
                index=['d', 'ARMA_base', 'Distribución', 'Varianza'],
                columns='Modalidad',
                values=model
            ).reset_index()
            
            detailed.columns.name = None
            detailed = detailed.rename(columns={
                'SIN_DIFF': 'ECRPS_Sin_Dif', 
                'CON_DIFF': 'ECRPS_Con_Dif'
            })
            
            detailed['Mejora_Absoluta'] = detailed['ECRPS_Sin_Dif'] - detailed['ECRPS_Con_Dif']
            detailed['Mejora_%'] = (detailed['Mejora_Absoluta'] / detailed['ECRPS_Sin_Dif']) * 100
            
            # Ordenar por d y luego por mejora
            detailed = detailed.sort_values(by=['d', 'Mejora_%'], ascending=[True, False])
            
            sheet_name = model[:31]
            detailed.to_excel(writer, sheet_name=sheet_name, index=False)
            
            # Formatos condicionales
            workbook = writer.book
            worksheet = writer.sheets[sheet_name]
            fmt_green = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
            fmt_red = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
            
            # Aplicar a columna Mejora_%
            col_idx = detailed.columns.get_loc('Mejora_%')
            worksheet.conditional_format(1, col_idx, len(detailed), col_idx, {
                'type': 'cell', 'criteria': '>', 'value': 0, 'format': fmt_green
            })
            worksheet.conditional_format(1, col_idx, len(detailed), col_idx, {
                'type': 'cell', 'criteria': '<', 'value': 0, 'format': fmt_red
            })

# ====================================================================================
# SECCIÓN ADICIONAL: GRÁFICOS PREVIOS MANTENIDOS
# ====================================================================================
def plot_mejora_general():
    """Gráfico general de mejora promedio por modelo (todos los d)"""
    print("\n=== Generando gráfico general de mejora ===")
    
    stats_list = []
    for model in model_cols:
        sin = df[df['Modalidad'] == 'SIN_DIFF'][model].mean()
        con = df[df['Modalidad'] == 'CON_DIFF'][model].mean()
        mejora = ((sin - con) / sin) * 100 if sin != 0 else 0
        stats_list.append({'Modelo': model, 'Mejora_%': mejora})
    
    df_stats = pd.DataFrame(stats_list).sort_values(by='Mejora_%', ascending=False)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    colors = ['#2ecc71' if m > 0 else '#e74c3c' for m in df_stats['Mejora_%']]
    bars = ax.barh(df_stats['Modelo'], df_stats['Mejora_%'], color=colors, edgecolor='black')
    ax.bar_label(bars, padding=5, fmt='%.1f%%', fontweight='bold')
    ax.axvline(0, color='black', lw=1)
    ax.set_title('Mejora Porcentual Global con Diferenciación (Todos los d)', fontweight='bold')
    ax.set_xlabel('Mejora (%)')
    plt.tight_layout()
    plt.savefig(plots_dir / '0_mejora_global.png')
    plt.close()

# ====================================================================================
# EJECUCIÓN PRINCIPAL
# ====================================================================================
if __name__ == "__main__":
    print("\n" + "="*80)
    print("INICIANDO ANÁLISIS MULTI-D CON DM TEST (FIXED-M ASYMPTOTICS)")
    print("="*80)
    
    plot_mejora_general()
    plot_individual_comparisons()
    plot_heatmap_d_vs_modelo()
    plot_heatmap_significancia()
    plot_interacciones_d()
    run_dm_analysis()
    generate_detailed_excel()
    
    print("\n" + "="*80)
    print(f"✓ ANÁLISIS COMPLETADO EXITOSAMENTE")
    print(f"✓ Resultados guardados en: {output_dir}")
    print(f"✓ Gráficos guardados en: {plots_dir}")
    print("="*80)

✓ Datos cargados: (26880, 19)
✓ Columnas: ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 'Distribución', 'Varianza', 'Modalidad', 'Valor_Observado', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
✓ Modelos identificados: ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
✓ Valores únicos de d: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(10)]

INICIANDO ANÁLISIS MULTI-D CON DM TEST (FIXED-M ASYMPTOTICS)

=== Generando gráfico general de mejora ===

=== Generando gráficos de barras con valores por d ===

=== Generando Heatmap d vs Modelo ===

=== Generando Heatmap de Significancia d vs Diferenciación (DM Test) ===

=== Generando Interacciones d vs Dist, Config y Varianza ===

=== Calculando Tabla DM por valor de d (Fixed-m Asymptotics) ===
✓ Tabla DM guardada con 72 comparaciones

=== Generando Excel detalla

## Aumento d ARIMA

### Pre-procesamiento

In [4]:
import pandas as pd
import numpy as np
from scipy import stats

# Cargar datos
base = pd.read_excel("./datos/Simulacion/Multi_D/resultados_ARIMA_d1_a_d10_DOBLE_MODALIDAD_COMPLETO.xlsx")

# Seleccionar columnas necesarias
columnas_seleccionadas = ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 
                          'Distribución', 'Varianza', 'Modalidad', 'Sieve Bootstrap']
datos = base[columnas_seleccionadas].copy()

# Renombrar columna para claridad
datos.rename(columns={'Sieve Bootstrap': 'ECRPS'}, inplace=True)

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano con fixed-smoothing asymptotics (Coroneo & Iacone, 2020)
    
    Implementa dos enfoques:
    1. Fixed-b asymptotics con kernel de Bartlett
    2. Fixed-m asymptotics con kernel de Daniell
    
    Parameters:
    -----------
    errors1, errors2 : array-like
        Errores de pronóstico (ECRPS) de los dos modelos
    h : int
        Horizonte de pronóstico (forecast horizon)
    
    Returns:
    --------
    hln_dm_stat : float
        Estadístico con fixed-smoothing asymptotics
    p_value : float
        P-valor usando distribución apropiada
    dm_stat : float
        Estadístico DM original (para referencia)
    """
    # Calcular diferencial de pérdida
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    if T < 2:
        return np.nan, np.nan, np.nan
    
    # Desviaciones de la media
    u = d - d_bar
    
    # Elegir método según tamaño de muestra
    if T >= 80:
        # Fixed-b con kernel de Bartlett para muestras medianas/grandes
        M = int(np.floor(T**(1/2)))  # Bandwidth recomendado
        b = M / T
        
        # Calcular WCE con kernel de Bartlett
        gamma_0 = np.mean(u**2)
        gamma_sum = gamma_0
        
        for j in range(1, min(M, T)):
            gamma_j = np.mean(u[j:] * u[:-j])
            weight = 1 - j/M  # Kernel de Bartlett
            gamma_sum += 2 * weight * gamma_j
        
        sigma_hat_sq = gamma_sum
        
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
        
        # Estadístico DM
        dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
        
        # Valor crítico fixed-b para Bartlett kernel (fórmula del paper)
        # Para test de dos colas al 5%
        alpha_0, alpha_1, alpha_2, alpha_3 = 1.9600, 2.9694, 0.4160, -0.5324
        critical_value = alpha_0 + alpha_1*b + alpha_2*b**2 + alpha_3*b**3
        
        # P-valor aproximado usando la distribución límite
        # Nota: esto es una aproximación, idealmente se simularía
        p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat) / (critical_value/1.96)))
        
        hln_dm_stat = dm_stat
        
    else:
        # Fixed-m con kernel de Daniell para muestras pequeñas
        m = int(np.floor(T**(1/3)))  # Bandwidth recomendado
        
        # Calcular periodograma
        from scipy.fft import fft
        
        # FFT de las desviaciones
        fft_u = fft(u)
        periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
        
        # WPE con kernel de Daniell (promedio de primeros m periodogramas)
        sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
        
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
        
        # Estadístico DM
        dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
        
        # Fixed-m asymptotics: límite es t-Student con 2m grados de libertad
        df = 2 * m
        hln_dm_stat = dm_stat
        
        # P-valor usando t-Student
        p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat

# Obtener valores únicos de d
valores_d = sorted(datos['d'].unique())

# Lista para almacenar resultados
resultados = []

# Iterar sobre cada valor de d
for d_val in valores_d:
    # Filtrar datos para el d actual
    datos_d = datos[datos['d'] == d_val].copy()
    
    # Separar por modalidad
    sin_diff = datos_d[datos_d['Modalidad'] == 'SIN_DIFF']['ECRPS'].values
    con_diff = datos_d[datos_d['Modalidad'] == 'CON_DIFF']['ECRPS'].values
    
    # Verificar que ambas modalidades tengan datos
    if len(sin_diff) == 0 or len(con_diff) == 0:
        print(f"Advertencia: d={d_val} no tiene datos para ambas modalidades")
        continue
    
    # Verificar que tengan la misma longitud
    if len(sin_diff) != len(con_diff):
        print(f"Advertencia: d={d_val} tiene diferente número de observaciones entre modalidades")
        min_len = min(len(sin_diff), len(con_diff))
        sin_diff = sin_diff[:min_len]
        con_diff = con_diff[:min_len]
    
    # Calcular estadísticas descriptivas
    ecrps_sin_diff_mean = np.mean(sin_diff)
    ecrps_con_diff_mean = np.mean(con_diff)
    diferencia = ecrps_sin_diff_mean - ecrps_con_diff_mean
    
    # Realizar test Diebold-Mariano modificado
    hln_dm_stat, p_value, dm_stat = modified_diebold_mariano_test(sin_diff, con_diff, h=1)
    
    # Determinar significancia (niveles comunes: 0.01, 0.05, 0.10)
    if p_value < 0.01:
        significativo = "***"
    elif p_value < 0.05:
        significativo = "**"
    elif p_value < 0.10:
        significativo = "*"
    else:
        significativo = "No"
    
    # Agregar a resultados
    resultados.append({
        'd': d_val,
        'N_obs': len(sin_diff),
        'ECRPS_SIN_DIFF': ecrps_sin_diff_mean,
        'ECRPS_CON_DIFF': ecrps_con_diff_mean,
        'Diferencia': diferencia,
        'DM_stat': dm_stat,
        'HLN-DM_stat': hln_dm_stat,
        'p_valor': p_value,
        'Significativo': significativo
    })

# Crear DataFrame con resultados
resultados_df = pd.DataFrame(resultados)

# Formatear para mejor visualización
resultados_df['ECRPS_SIN_DIFF'] = resultados_df['ECRPS_SIN_DIFF'].round(6)
resultados_df['ECRPS_CON_DIFF'] = resultados_df['ECRPS_CON_DIFF'].round(6)
resultados_df['Diferencia'] = resultados_df['Diferencia'].round(6)
resultados_df['DM_stat'] = resultados_df['DM_stat'].round(4)
resultados_df['HLN-DM_stat'] = resultados_df['HLN-DM_stat'].round(4)
resultados_df['p_valor'] = resultados_df['p_valor'].round(4)

# Mostrar resultados
print("\n" + "="*100)
print("TEST DIEBOLD-MARIANO MODIFICADO (HLN): Comparación SIN_DIFF vs CON_DIFF por valor de d")
print("="*100)
print("\nH0: No hay diferencia significativa entre las modalidades")
print("H1: Hay diferencia significativa entre las modalidades")
print("\nSignificancia: *** p<0.01, ** p<0.05, * p<0.10, No = no significativo")
print("\n")
print(resultados_df.to_string(index=False))

# Resumen de resultados
print("\n" + "="*100)
print("RESUMEN")
print("="*100)
n_significativos = len(resultados_df[resultados_df['Significativo'] != 'No'])
n_total = len(resultados_df)
print(f"\nTotal de comparaciones: {n_total}")
print(f"Diferencias significativas: {n_significativos} ({100*n_significativos/n_total:.1f}%)")
print(f"No significativas: {n_total - n_significativos} ({100*(n_total-n_significativos)/n_total:.1f}%)")



TEST DIEBOLD-MARIANO MODIFICADO (HLN): Comparación SIN_DIFF vs CON_DIFF por valor de d

H0: No hay diferencia significativa entre las modalidades
H1: Hay diferencia significativa entre las modalidades

Significancia: *** p<0.01, ** p<0.05, * p<0.10, No = no significativo


 d  N_obs  ECRPS_SIN_DIFF  ECRPS_CON_DIFF    Diferencia  DM_stat  HLN-DM_stat  p_valor Significativo
 1   1680    5.495070e-01    5.485800e-01  9.280000e-04   0.5334       0.5334   0.6067            No
 2   1680    2.884661e+01    2.883279e+01  1.382000e-02   0.8679       0.8679   0.4022            No
 3   1680    2.725984e+01    2.726032e+01 -4.790000e-04  -0.0186      -0.0186   0.9856            No
 4   1680    2.704997e+01    2.704544e+01  4.532000e-03   0.1216       0.1216   0.9066            No
 5   1680    3.451632e+01    2.821125e+01  6.305071e+00   1.8941       1.8941   0.0676             *
 6   1680    1.400979e+03    2.663349e+01  1.374346e+03   4.4940       4.4940   0.0000           ***
 7   1680    9.737

In [3]:
import pandas as pd
import numpy as np
from scipy import stats

# Cargar datos
base = pd.read_excel("./datos/Simulacion/Diferenciado/resultados_UNION_ARIMA.xlsx")

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano con fixed-smoothing asymptotics (Coroneo & Iacone, 2020)
    
    Implementa dos enfoques:
    1. Fixed-b asymptotics con kernel de Bartlett
    2. Fixed-m asymptotics con kernel de Daniell
    
    Parameters:
    -----------
    errors1, errors2 : array-like
        Errores de pronóstico (ECRPS) de los dos modelos
    h : int
        Horizonte de pronóstico (forecast horizon)
    
    Returns:
    --------
    hln_dm_stat : float
        Estadístico con fixed-smoothing asymptotics
    p_value : float
        P-valor usando distribución apropiada
    dm_stat : float
        Estadístico DM original (para referencia)
    """
    # Calcular diferencial de pérdida
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    if T < 2:
        return np.nan, np.nan, np.nan
    
    # Desviaciones de la media
    u = d - d_bar
    
    # Elegir método según tamaño de muestra
    if T >= 80:
        # Fixed-b con kernel de Bartlett para muestras medianas/grandes
        M = int(np.floor(T**(1/2)))  # Bandwidth recomendado
        b = M / T
        
        # Calcular WCE con kernel de Bartlett
        gamma_0 = np.mean(u**2)
        gamma_sum = gamma_0
        
        for j in range(1, min(M, T)):
            gamma_j = np.mean(u[j:] * u[:-j])
            weight = 1 - j/M  # Kernel de Bartlett
            gamma_sum += 2 * weight * gamma_j
        
        sigma_hat_sq = gamma_sum
        
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
        
        # Estadístico DM
        dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
        
        # Valor crítico fixed-b para Bartlett kernel (fórmula del paper)
        # Para test de dos colas al 5%
        alpha_0, alpha_1, alpha_2, alpha_3 = 1.9600, 2.9694, 0.4160, -0.5324
        critical_value = alpha_0 + alpha_1*b + alpha_2*b**2 + alpha_3*b**3
        
        # P-valor aproximado usando la distribución límite
        # Nota: esto es una aproximación, idealmente se simularía
        p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat) / (critical_value/1.96)))
        
        hln_dm_stat = dm_stat
        
    else:
        # Fixed-m con kernel de Daniell para muestras pequeñas
        m = int(np.floor(T**(1/3)))  # Bandwidth recomendado
        
        # Calcular periodograma
        from scipy.fft import fft
        
        # FFT de las desviaciones
        fft_u = fft(u)
        periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
        
        # WPE con kernel de Daniell (promedio de primeros m periodogramas)
        sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
        
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
        
        # Estadístico DM
        dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
        
        # Fixed-m asymptotics: límite es t-Student con 2m grados de libertad
        df = 2 * m
        hln_dm_stat = dm_stat
        
        # P-valor usando t-Student
        p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat


# Separar por diferenciación
sin_diff = base[base['Diferenciacion'] == 'No'].copy()
con_diff = base[base['Diferenciacion'] == 'Si'].copy()

# Métodos a comparar (todos excepto las columnas de configuración)
metodos = ['Block Bootstrapping', 'Sieve Bootstrap', 'LSPM', 'LSPMW', 
           'AREPD', 'MCPS', 'AV-MCPS', 'DeepAR', 'EnCQR-LSTM']

# Lista para almacenar resultados
resultados = []

print("\n" + "="*120)
print("TEST DIEBOLD-MARIANO MODIFICADO (HLN): Comparación Sin Diferenciación (No) vs Con Diferenciación (Si)")
print("="*120)
print("\nH0: No hay diferencia significativa entre aplicar o no diferenciación")
print("H1: Hay diferencia significativa entre aplicar o no diferenciación")
print("\nSignificancia: *** p<0.01, ** p<0.05, * p<0.10, No = no significativo")
print("="*120)

# Iterar sobre cada método
for metodo in metodos:
    # Obtener valores de ECRPS para cada modalidad
    ecrps_sin = sin_diff[metodo].values
    ecrps_con = con_diff[metodo].values
    
    # Verificar que ambas modalidades tengan datos
    if len(ecrps_sin) == 0 or len(ecrps_con) == 0:
        print(f"\nAdvertencia: {metodo} no tiene datos para ambas modalidades")
        continue
    
    # Verificar que tengan la misma longitud
    if len(ecrps_sin) != len(ecrps_con):
        print(f"\nAdvertencia: {metodo} tiene diferente número de observaciones")
        min_len = min(len(ecrps_sin), len(ecrps_con))
        ecrps_sin = ecrps_sin[:min_len]
        ecrps_con = ecrps_con[:min_len]
    
    # Calcular estadísticas descriptivas
    ecrps_sin_mean = np.mean(ecrps_sin)
    ecrps_con_mean = np.mean(ecrps_con)
    diferencia = ecrps_sin_mean - ecrps_con_mean
    
    # Realizar test Diebold-Mariano modificado
    hln_dm_stat, p_value, dm_stat = modified_diebold_mariano_test(ecrps_sin, ecrps_con, h=1)
    
    # Determinar significancia
    if p_value < 0.01:
        significativo = "***"
    elif p_value < 0.05:
        significativo = "**"
    elif p_value < 0.10:
        significativo = "*"
    else:
        significativo = "No"
    
    # Agregar a resultados
    resultados.append({
        'Método': metodo,
        'N_obs': len(ecrps_sin),
        'ECRPS_Sin_Diff': ecrps_sin_mean,
        'ECRPS_Con_Diff': ecrps_con_mean,
        'Diferencia': diferencia,
        'DM_stat': dm_stat,
        'HLN-DM_stat': hln_dm_stat,
        'p_valor': p_value,
        'Significativo': significativo
    })

# Crear DataFrame con resultados
resultados_df = pd.DataFrame(resultados)

# Formatear para mejor visualización
resultados_df['ECRPS_Sin_Diff'] = resultados_df['ECRPS_Sin_Diff'].round(6)
resultados_df['ECRPS_Con_Diff'] = resultados_df['ECRPS_Con_Diff'].round(6)
resultados_df['Diferencia'] = resultados_df['Diferencia'].round(6)
resultados_df['DM_stat'] = resultados_df['DM_stat'].round(4)
resultados_df['HLN-DM_stat'] = resultados_df['HLN-DM_stat'].round(4)
resultados_df['p_valor'] = resultados_df['p_valor'].round(4)

# Mostrar resultados
print("\n")
print(resultados_df.to_string(index=False))

# Resumen de resultados
print("\n" + "="*120)
print("RESUMEN")
print("="*120)
n_significativos = len(resultados_df[resultados_df['Significativo'] != 'No'])
n_total = len(resultados_df)
print(f"\nTotal de métodos comparados: {n_total}")
print(f"Diferencias significativas: {n_significativos} ({100*n_significativos/n_total:.1f}%)")
print(f"No significativas: {n_total - n_significativos} ({100*(n_total-n_significativos)/n_total:.1f}%)")

# Mostrar cuáles métodos tienen diferencias significativas
if n_significativos > 0:
    print("\nMétodos con diferencias significativas:")
    metodos_sig = resultados_df[resultados_df['Significativo'] != 'No'][['Método', 'Significativo', 'p_valor']]
    print(metodos_sig.to_string(index=False))


TEST DIEBOLD-MARIANO MODIFICADO (HLN): Comparación Sin Diferenciación (No) vs Con Diferenciación (Si)

H0: No hay diferencia significativa entre aplicar o no diferenciación
H1: Hay diferencia significativa entre aplicar o no diferenciación

Significancia: *** p<0.01, ** p<0.05, * p<0.10, No = no significativo


             Método  N_obs  ECRPS_Sin_Diff  ECRPS_Con_Diff  Diferencia  DM_stat  HLN-DM_stat  p_valor Significativo
Block Bootstrapping   1680       11.251601        0.666133   10.585468   5.9736       5.9736   0.0000           ***
    Sieve Bootstrap   1680        0.547481        0.546020    0.001461   1.1882       1.1882   0.2515            No
               LSPM   1680        1.064804        0.648039    0.416766  14.2307      14.2307   0.0000           ***
              LSPMW   1680        3.079645        0.767172    2.312473  10.3870      10.3870   0.0000           ***
              AREPD   1680       10.031183        0.704149    9.327035   5.7364       5.7364   0.0000     

### Analisis general

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
import warnings

# Configuración inicial
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'

# ====================================================================================
# 1. PREPARACIÓN DE DIRECTORIOS
# ====================================================================================
output_dir = Path("./Resultados_analisis/Multi_d")
output_dir.mkdir(parents=True, exist_ok=True)
plots_dir = output_dir / "Graficos_Analisis"
plots_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
path_excel = "./datos/Simulacion/Multi_D/resultados_ARIMA_d1_a_d10_DOBLE_MODALIDAD_COMPLETO.xlsx"
try:
    df = pd.read_excel(path_excel)
    print(f"✓ Datos cargados: {df.shape}")
    print(f"✓ Columnas: {df.columns.tolist()}")
except Exception as e:
    print(f"Error crítico: {e}")
    exit()

# Identificar columnas
var_cols = ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 'Distribución', 
            'Varianza', 'Modalidad', 'Valor_Observado']
model_cols = [col for col in df.columns if col not in var_cols]
print(f"✓ Modelos identificados: {model_cols}")
print(f"✓ Valores únicos de d: {sorted(df['d'].unique())}")

# ====================================================================================
# TEST DE DIEBOLD-MARIANO CON CORRECCIÓN
# ====================================================================================
def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano con fixed-smoothing asymptotics (Coroneo & Iacone, 2020)
    
    Usa Fixed-m asymptotics (kernel Daniell) que es más robusto en muestras pequeñas
    y mantiene buen desempeño en muestras grandes.
    
    Parameters:
    -----------
    errors1, errors2 : array-like
        Errores de pronóstico (ECRPS) de los dos modelos
    h : int
        Horizonte de pronóstico (forecast horizon)
    
    Returns:
    --------
    hln_dm_stat : float
        Estadístico con fixed-m asymptotics
    p_value : float
        P-valor usando distribución t-Student con 2m grados de libertad
    dm_stat : float
        Estadístico DM original (para referencia)
    """
    # Calcular diferencial de pérdida
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    if T < 2:
        return np.nan, np.nan, np.nan
    
    # Desviaciones de la media
    u = d - d_bar
    
    # Fixed-m: bandwidth recomendado en el paper
    m = max(1, int(np.floor(T**(1/3))))
    
    # Calcular periodograma
    from scipy.fft import fft
    
    # FFT de las desviaciones
    fft_u = fft(u)
    periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
    
    # WPE con kernel de Daniell: promedio de primeros m periodogramas
    # (excluyendo frecuencia 0)
    if m >= len(periodogram) - 1:
        m = len(periodogram) - 2
    
    sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
    
    if sigma_hat_sq <= 0:
        # Fallback: usar varianza simple
        sigma_hat_sq = np.var(d, ddof=1) / T
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
    
    # Estadístico DM
    dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
    
    # Fixed-m asymptotics: límite es t-Student con 2m grados de libertad
    df = 2 * m
    hln_dm_stat = dm_stat
    
    # P-valor usando t-Student
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat

# ====================================================================================
# NUEVO 1: HEATMAPS ECRPS POR MODALIDAD
# ====================================================================================
def plot_heatmaps_ecrps_por_modalidad():
    print("\n=== NUEVO 1: Heatmaps ECRPS por Modalidad ===")
    d_values = sorted(df['d'].unique())
    
    for modalidad in ['SIN_DIFF', 'CON_DIFF']:
        matrix_data = []
        for model in model_cols:
            row = []
            for d_val in d_values:
                subset = df[(df['d'] == d_val) & (df['Modalidad'] == modalidad)]
                avg_ecrps = subset[model].mean()
                row.append(avg_ecrps)
            matrix_data.append(row)
        
        df_heatmap = pd.DataFrame(matrix_data, index=model_cols, columns=[f'd={d}' for d in d_values])
        
        plt.figure(figsize=(14, 10))
        annot_matrix = df_heatmap.applymap(lambda x: f'{x:.2e}' if abs(x) >= 1000 else f'{x:.2f}')
        sns.heatmap(df_heatmap, annot=annot_matrix, fmt='', cmap='RdYlGn_r', 
                   cbar_kws={'label': 'ECRPS'}, linewidths=0.5, center=df_heatmap.mean().mean())
        plt.title(f'Heatmap ECRPS - Modalidad: {modalidad}', fontweight='bold', fontsize=14)
        plt.xlabel('Orden de Diferenciación (d)', fontweight='bold')
        plt.ylabel('Modelo', fontweight='bold')
        plt.tight_layout()
        plt.savefig(plots_dir / f'NUEVO_1_heatmap_ecrps_{modalidad}.png')
        plt.close()
        df_heatmap.to_excel(output_dir / f'NUEVO_1_tabla_ecrps_{modalidad}.xlsx')

# ====================================================================================
# NUEVO 2: SENSIBILIDAD AL INCREMENTO DE d
# ====================================================================================
def plot_sensibilidad_incremento_d():
    print("\n=== NUEVO 2: Sensibilidad al Incremento de d ===")
    d_values = sorted(df['d'].unique())
    sensibilidad = []
    
    for model in model_cols:
        cambios = []
        for i in range(len(d_values) - 1):
            avg_d1 = df[df['d'] == d_values[i]][model].mean()
            avg_d2 = df[df['d'] == d_values[i+1]][model].mean()
            cambios.append(abs(avg_d2 - avg_d1))
        
        sensibilidad_modelo = np.mean(cambios)
        std_entre_d = np.std([df[df['d'] == d][model].mean() for d in d_values])
        sensibilidad.append({'Modelo': model, 'Sensibilidad_Media': sensibilidad_modelo, 
                           'Std_entre_d': std_entre_d})
    
    df_sens = pd.DataFrame(sensibilidad).sort_values('Sensibilidad_Media', ascending=False)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    colors = plt.cm.RdYlGn_r(np.linspace(0.3, 0.9, len(df_sens)))
    
    bars1 = ax1.barh(df_sens['Modelo'], df_sens['Sensibilidad_Media'], color=colors, edgecolor='black')
    ax1.bar_label(bars1, fmt='%.3f', padding=5, fontsize=8)
    ax1.set_xlabel('Sensibilidad Media', fontweight='bold')
    ax1.set_title('Sensibilidad por Cambio Promedio', fontweight='bold')
    ax1.grid(axis='x', alpha=0.3)
    
    bars2 = ax2.barh(df_sens['Modelo'], df_sens['Std_entre_d'], color=colors, edgecolor='black')
    ax2.bar_label(bars2, fmt='%.3f', padding=5, fontsize=8)
    ax2.set_xlabel('Desviación Estándar', fontweight='bold')
    ax2.set_title('Sensibilidad por Variabilidad', fontweight='bold')
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(plots_dir / 'NUEVO_2_sensibilidad_incremento_d.png')
    plt.close()
    df_sens.to_excel(output_dir / 'NUEVO_2_tabla_sensibilidad.xlsx', index=False)
    
    # Trayectorias
    plt.figure(figsize=(14, 8))
    for model in model_cols:
        trayectoria = [df[df['d'] == d][model].mean() for d in d_values]
        plt.plot(d_values, trayectoria, marker='o', label=model, linewidth=2, alpha=0.7)
    plt.xlabel('Orden de Diferenciación (d)', fontweight='bold', fontsize=12)
    plt.ylabel('ECRPS Promedio', fontweight='bold', fontsize=12)
    plt.title('Trayectorias de ECRPS según d', fontweight='bold', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(plots_dir / 'NUEVO_2_trayectorias_d.png')
    plt.close()

# ====================================================================================
# NUEVO 3: TEST DIEBOLD-MARIANO
# ====================================================================================
def plot_heatmap_diebold_mariano():
    print("\n=== NUEVO 3: Test Diebold-Mariano ===")
    d_values = sorted(df['d'].unique())
    results = []
    
    for model in model_cols:
        for d_val in d_values:
            sin_diff = df[(df['d'] == d_val) & (df['Modalidad'] == 'SIN_DIFF')][model].values
            con_diff = df[(df['d'] == d_val) & (df['Modalidad'] == 'CON_DIFF')][model].values
            
            if len(sin_diff) > 5 and len(con_diff) > 5:
                dm_stat, p_value, _ = modified_diebold_mariano_test(sin_diff, con_diff)
                results.append({'Modelo': model, 'd': d_val, 'DM_stat': dm_stat, 
                              'p_valor': p_value, 'Significativo': 'Sí' if p_value < 0.05 else 'No'})
    
    df_dm = pd.DataFrame(results)
    pivot_pvalor = df_dm.pivot(index='Modelo', columns='d', values='p_valor')
    pivot_dm_stat = df_dm.pivot(index='Modelo', columns='d', values='DM_stat')
    
    # Heatmap p-valores
    plt.figure(figsize=(14, 10))
    sns.heatmap(pivot_pvalor, annot=True, fmt='.3f', cmap='RdYlGn', 
               cbar_kws={'label': 'p-valor'}, linewidths=0.5, vmin=0, vmax=0.1, center=0.05)
    plt.title('Test DM: P-valores (SIN_DIFF vs CON_DIFF)', fontweight='bold', fontsize=14)
    plt.xlabel('Orden de Diferenciación (d)', fontweight='bold')
    plt.ylabel('Modelo', fontweight='bold')
    plt.figtext(0.5, -0.02, 'Valores < 0.05: diferencias significativas', 
               ha='center', fontsize=10, style='italic')
    plt.tight_layout()
    plt.savefig(plots_dir / 'NUEVO_3_heatmap_dm_pvalor.png', bbox_inches='tight')
    plt.close()
    
    # Heatmap estadísticos
    plt.figure(figsize=(14, 10))
    sns.heatmap(pivot_dm_stat, annot=True, fmt='.2f', cmap='RdBu_r', 
               cbar_kws={'label': 'Estadístico DM'}, linewidths=0.5, center=0)
    plt.title('Test DM: Estadísticos', fontweight='bold', fontsize=14)
    plt.xlabel('Orden de Diferenciación (d)', fontweight='bold')
    plt.ylabel('Modelo', fontweight='bold')
    plt.tight_layout()
    plt.savefig(plots_dir / 'NUEVO_3_heatmap_dm_stat.png', bbox_inches='tight')
    plt.close()
    
    df_dm.to_excel(output_dir / 'NUEVO_3_tabla_diebold_mariano.xlsx', index=False)
    pivot_pvalor.to_excel(output_dir / 'NUEVO_3_matriz_pvalores.xlsx')

# ====================================================================================
# MODIFICADO 1.2: HEATMAPS POR d
# ====================================================================================
def plot_heatmaps_mejora_por_d():
    print("\n=== MODIFICADO 1.2: Heatmaps por d ===")
    d_values = sorted(df['d'].unique())
    
    for d_val in d_values:
        subset_d = df[df['d'] == d_val]
        mejoras = []
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].mean()
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].mean()
            mejora_pct = ((sin - con) / sin) * 100 if sin != 0 else 0
            mejoras.append(mejora_pct)
        
        df_mejora = pd.DataFrame([mejoras], columns=model_cols, index=[f'd={d_val}'])
        
        plt.figure(figsize=(16, 3))
        sns.heatmap(df_mejora, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
                   cbar_kws={'label': 'Mejora (%)'}, linewidths=1)
        plt.title(f'Mejora (CON_DIFF vs SIN_DIFF) - d={d_val}', fontweight='bold')
        plt.xlabel('Modelo', fontweight='bold')
        plt.tight_layout()
        plt.savefig(plots_dir / f'MOD_1.2_mejora_d{d_val}.png')
        plt.close()

# ====================================================================================
# MODIFICADO 3: HEATMAPS POR PROCESO
# ====================================================================================
def plot_heatmaps_mejora_por_proceso():
    print("\n=== MODIFICADO 3: Heatmaps por Proceso ===")
    d_values = sorted(df['d'].unique())
    
    for model in model_cols:
        mejoras_proceso = []
        procesos = sorted(df['Proceso'].unique())
        
        for proceso in procesos:
            mejoras_d = []
            for d_val in d_values:
                subset = df[(df['d'] == d_val) & (df['Proceso'] == proceso)]
                sin = subset[subset['Modalidad'] == 'SIN_DIFF'][model].mean()
                con = subset[subset['Modalidad'] == 'CON_DIFF'][model].mean()
                mejora_pct = ((sin - con) / sin) * 100 if sin != 0 else 0
                mejoras_d.append(mejora_pct)
            mejoras_proceso.append(mejoras_d)
        
        df_mejora = pd.DataFrame(mejoras_proceso, index=procesos, 
                                columns=[f'd={d}' for d in d_values])
        
        plt.figure(figsize=(14, max(8, len(procesos) * 0.5)))
        sns.heatmap(df_mejora, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
                   cbar_kws={'label': 'Mejora (%)'}, linewidths=0.5)
        plt.title(f'Mejora por Proceso - {model}', fontweight='bold')
        plt.xlabel('Orden de Diferenciación (d)', fontweight='bold')
        plt.ylabel('Proceso ARMA', fontweight='bold')
        plt.tight_layout()
        
        safe_name = model.replace(' ', '_').replace('/', '_')
        plt.savefig(plots_dir / f'MOD_3_proceso_{safe_name}.png')
        plt.close()
        df_mejora.to_excel(output_dir / f'MOD_3_proceso_{safe_name}.xlsx')

# ====================================================================================
# MODIFICADO 4: HEATMAPS POR DISTRIBUCIÓN
# ====================================================================================
def plot_heatmaps_mejora_por_distribucion():
    print("\n=== MODIFICADO 4: Heatmaps por Distribución ===")
    d_values = sorted(df['d'].unique())
    
    for model in model_cols:
        mejoras_dist = []
        distribuciones = sorted(df['Distribución'].unique())
        
        for dist in distribuciones:
            mejoras_d = []
            for d_val in d_values:
                subset = df[(df['d'] == d_val) & (df['Distribución'] == dist)]
                sin = subset[subset['Modalidad'] == 'SIN_DIFF'][model].mean()
                con = subset[subset['Modalidad'] == 'CON_DIFF'][model].mean()
                mejora_pct = ((sin - con) / sin) * 100 if sin != 0 else 0
                mejoras_d.append(mejora_pct)
            mejoras_dist.append(mejoras_d)
        
        df_mejora = pd.DataFrame(mejoras_dist, index=distribuciones, 
                                columns=[f'd={d}' for d in d_values])
        
        plt.figure(figsize=(14, max(6, len(distribuciones) * 1.5)))
        sns.heatmap(df_mejora, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
                   cbar_kws={'label': 'Mejora (%)'}, linewidths=0.5)
        plt.title(f'Mejora por Distribución - {model}', fontweight='bold')
        plt.xlabel('d', fontweight='bold')
        plt.ylabel('Distribución', fontweight='bold')
        plt.tight_layout()
        
        safe_name = model.replace(' ', '_').replace('/', '_')
        plt.savefig(plots_dir / f'MOD_4_dist_{safe_name}.png')
        plt.close()
        df_mejora.to_excel(output_dir / f'MOD_4_dist_{safe_name}.xlsx')

# ====================================================================================
# MODIFICADO 5: MATRICES VARIANZA
# ====================================================================================
def plot_matrices_mejora_por_varianza():
    print("\n=== MODIFICADO 5: Matrices Varianza ===")
    d_values = sorted(df['d'].unique())
    varianzas = sorted(df['Varianza'].unique())
    
    for model in model_cols:
        mejoras_var = []
        for d_val in d_values:
            mejoras_por_var = []
            for var in varianzas:
                subset = df[(df['d'] == d_val) & (df['Varianza'] == var)]
                sin = subset[subset['Modalidad'] == 'SIN_DIFF'][model].mean()
                con = subset[subset['Modalidad'] == 'CON_DIFF'][model].mean()
                mejora_pct = ((sin - con) / sin) * 100 if sin != 0 else 0
                mejoras_por_var.append(mejora_pct)
            mejoras_var.append(mejoras_por_var)
        
        df_mejora = pd.DataFrame(mejoras_var, index=[f'd={d}' for d in d_values], 
                                columns=[f'Var={v}' for v in varianzas])
        
        plt.figure(figsize=(max(10, len(varianzas) * 2), 10))
        sns.heatmap(df_mejora, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
                   cbar_kws={'label': 'Mejora (%)'}, linewidths=0.5)
        plt.title(f'Mejora por Varianza - {model}', fontweight='bold')
        plt.xlabel('Varianza', fontweight='bold')
        plt.ylabel('d', fontweight='bold')
        plt.tight_layout()
        
        safe_name = model.replace(' ', '_').replace('/', '_')
        plt.savefig(plots_dir / f'MOD_5_var_{safe_name}.png')
        plt.close()
        df_mejora.to_excel(output_dir / f'MOD_5_var_{safe_name}.xlsx')

# ====================================================================================
# MODIFICADO 9: EXCEL DETALLADO CON d
# ====================================================================================
def generate_detailed_excel_with_d():
    print("\n=== MODIFICADO 9: Excel con d ===")
    file_path = output_dir / "MOD_9_Detallado_Con_d.xlsx"
    
    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        for model in model_cols:
            temp = df.groupby(['d', 'Proceso', 'ARMA_base', 'Distribución', 
                              'Varianza', 'Modalidad'])[model].mean().reset_index()
            
            detailed = temp.pivot_table(
                index=['d', 'Proceso', 'ARMA_base', 'Distribución', 'Varianza'],
                columns='Modalidad', values=model).reset_index()
            
            detailed.columns.name = None
            detailed = detailed.rename(columns={'SIN_DIFF': 'ECRPS_Sin_Dif', 
                                               'CON_DIFF': 'ECRPS_Con_Dif'})
            
            detailed['Mejora_Absoluta'] = detailed['ECRPS_Sin_Dif'] - detailed['ECRPS_Con_Dif']
            detailed['Mejora_%'] = (detailed['Mejora_Absoluta'] / detailed['ECRPS_Sin_Dif']) * 100
            detailed = detailed.sort_values(by=['d', 'Mejora_%'], ascending=[True, False])
            
            sheet_name = model.replace(' ', '_').replace('/', '_')[:31]
            detailed.to_excel(writer, sheet_name=sheet_name, index=False)
            
            workbook = writer.book
            worksheet = writer.sheets[sheet_name]
            fmt_green = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
            fmt_red = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
            
            col_idx = detailed.columns.get_loc('Mejora_%')
            worksheet.conditional_format(1, col_idx, len(detailed), col_idx, 
                                       {'type': 'cell', 'criteria': '>', 'value': 0, 'format': fmt_green})
            worksheet.conditional_format(1, col_idx, len(detailed), col_idx,
                                       {'type': 'cell', 'criteria': '<', 'value': 0, 'format': fmt_red})

# ====================================================================================
# FUNCIONES ORIGINALES
# ====================================================================================
def plot_heatmap_d_vs_modelo():
    print("\n=== Heatmap d vs Modelo ===")
    results = []
    d_values = sorted(df['d'].unique())
    
    for d_val in d_values:
        subset_d = df[df['d'] == d_val]
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].mean()
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].mean()
            mejora = ((sin - con) / sin) * 100 if sin != 0 else 0
            results.append({'d': d_val, 'Modelo': model, 'Mejora_%': mejora})
    
    df_mejora = pd.DataFrame(results)
    pivot = df_mejora.pivot(index='Modelo', columns='d', values='Mejora_%')
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(pivot, annot=True, fmt=".1f", cmap="RdYlGn", center=0, 
                cbar_kws={'label': 'Mejora (%)'}, linewidths=0.5)
    plt.title("Mejora: Modelo vs d", fontweight='bold', fontsize=14)
    plt.xlabel("Valor de d", fontweight='bold')
    plt.ylabel("Modelo", fontweight='bold')
    plt.tight_layout()
    plt.savefig(plots_dir / '2_heatmap_d_vs_modelo.png')
    plt.close()
    pivot.to_excel(output_dir / '2_tabla_d_vs_modelo.xlsx')

# ====================================================================================
# EJECUCIÓN PRINCIPAL
# ====================================================================================
if __name__ == "__main__":
    print("\n" + "="*80)
    print("INICIANDO ANÁLISIS MULTI-D COMPLETO")
    print("="*80)
    
    # Nuevas funciones
    plot_heatmaps_ecrps_por_modalidad()
    plot_sensibilidad_incremento_d()
    plot_heatmap_diebold_mariano()
    
    # Modificadas
    plot_heatmaps_mejora_por_d()
    plot_heatmaps_mejora_por_proceso()
    plot_heatmaps_mejora_por_distribucion()
    plot_matrices_mejora_por_varianza()
    generate_detailed_excel_with_d()
    
    # Originales
    plot_heatmap_d_vs_modelo()
    
    print("\n" + "="*80)
    print(f"✓ ANÁLISIS COMPLETADO")
    print(f"✓ Resultados: {output_dir}")
    print(f"✓ Gráficos: {plots_dir}")
    print("="*80)

✓ Datos cargados: (26880, 19)
✓ Columnas: ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 'Distribución', 'Varianza', 'Modalidad', 'Valor_Observado', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
✓ Modelos identificados: ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
✓ Valores únicos de d: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(10)]

INICIANDO ANÁLISIS MULTI-D COMPLETO

=== NUEVO 1: Heatmaps ECRPS por Modalidad ===

=== NUEVO 2: Sensibilidad al Incremento de d ===

=== NUEVO 3: Test Diebold-Mariano ===

=== MODIFICADO 1.2: Heatmaps por d ===

=== MODIFICADO 3: Heatmaps por Proceso ===

=== MODIFICADO 4: Heatmaps por Distribución ===

=== MODIFICADO 5: Matrices Varianza ===

=== MODIFICADO 9: Excel con d ===

=== Heatmap d vs Modelo ===

✓ ANÁLISIS COMPLETADO
✓ Resultados: Resultados_analisis\Multi

## Analisis Tamaño

### Pre-procesamiento

In [6]:
import pandas as pd
import numpy as np

# 1. Leer los tres archivos
arma_df = pd.read_excel("./datos/Simulacion/Tamaño/resultados_TAMANOS_CRECIENTES_ARMA.xlsx")
arima_df = pd.read_excel("./datos/Simulacion/Tamaño/resultados_TAMANOS_CRECIENTES_ARIMA.xlsx")
setar_df = pd.read_excel("./datos/Simulacion/Tamaño/resultados_TAMANOS_CRECIENTES_SETAR.xlsx")

# 2. ESTANDARIZAR NOMBRES DE COLUMNAS
# Renombramos 'Block Bootstrap' a 'Block Bootstrapping' en ARIMA y SETAR para que coincidan con ARMA
arima_df = arima_df.rename(columns={'Block Bootstrap': 'Block Bootstrapping'})
setar_df = setar_df.rename(columns={'Block Bootstrap': 'Block Bootstrapping'})

# 3. Asignar escenarios
arma_df['ESCENARIO'] = "Lineal Estacionario"
arima_df['ESCENARIO'] = "Lineal No estacionario"
setar_df['ESCENARIO'] = "No lineal Estacionario"

# 4. Filtrar los que no tienen "Promedio" en la columna "Paso"
arma_df = arma_df[arma_df['Paso'] != 'Promedio']
arima_df = arima_df[arima_df['Paso'] != 'Promedio']
setar_df = setar_df[setar_df['Paso'] != 'Promedio']

# 5. Lista de modelos (ahora el nombre es uniforme)
modelos = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# 6. Crear tabla comparativa
comparacion = []

for modelo in modelos:
    fila = {'Modelo': modelo}
    
    # Calcular promedio para cada escenario
    arma_promedio = arma_df[modelo].mean() if modelo in arma_df.columns else np.nan
    arima_promedio = arima_df[modelo].mean() if modelo in arima_df.columns else np.nan
    setar_promedio = setar_df[modelo].mean() if modelo in setar_df.columns else np.nan
    
    fila['ARMA'] = arma_promedio
    fila['ARIMA'] = arima_promedio
    fila['SETAR'] = setar_promedio
    
    # Determinar mejor escenario (menor promedio)
    promedios = {
        'ARMA': arma_promedio,
        'ARIMA': arima_promedio,
        'SETAR': setar_promedio
    }
    
    promedios_validos = {k: v for k, v in promedios.items() if not pd.isna(v)}
    
    if promedios_validos:
        mejor_escenario = min(promedios_validos, key=promedios_validos.get)
        fila['Mejor_Escenario'] = mejor_escenario
    else:
        fila['Mejor_Escenario'] = 'N/A'
    
    comparacion.append(fila)

# Crear DataFrame con la tabla comparativa
tabla_comparativa = pd.DataFrame(comparacion)
columnas_numericas = ['ARMA', 'ARIMA', 'SETAR']
tabla_comparativa[columnas_numericas] = tabla_comparativa[columnas_numericas].round(4)

# Mostrar tabla comparativa
print("\n" + "="*80)
print("TABLA COMPARATIVA DE MODELOS POR ESCENARIO")
print("(Promedio de amplitud de intervalos de predicción)")
print("="*80)
print(tabla_comparativa.to_string(index=False))
print("="*80 + "\n")

# 7. Procesamiento final y exportación
if 'Descripción' in setar_df.columns:
    setar_df = setar_df.drop('Descripción', axis=1)

# Concatenar los tres dataframes (ahora las columnas se alinearán perfectamente)
base_consolidada = pd.concat([arma_df, arima_df, setar_df], ignore_index=True)

# Guardar en un archivo Excel
base_consolidada.to_excel("./datos/Simulacion/Tamaño/Base_Tamaño_3_escenarios.xlsx", index=False)

print("\nArchivo 'Base_Tamaño_3_escenarios.xlsx' creado exitosamente!")
print(f"Columna 'Block Bootstrapping' verificada en el archivo final.")
print(f"Total de filas: {len(base_consolidada)}")


TABLA COMPARATIVA DE MODELOS POR ESCENARIO
(Promedio de amplitud de intervalos de predicción)
             Modelo   ARMA   ARIMA  SETAR Mejor_Escenario
              AREPD 0.9264 12.3986 0.8148           SETAR
            AV-MCPS 0.6780  3.0524 0.6585           SETAR
Block Bootstrapping 0.9036 14.2503 0.7915           SETAR
             DeepAR 0.5745  2.8056 0.5736           SETAR
         EnCQR-LSTM 0.7947  6.1857 0.7544           SETAR
               LSPM 0.7721  1.0854 0.6490           SETAR
              LSPMW 0.7839  1.0846 0.6610           SETAR
               MCPS 0.6613  2.8912 0.6488           SETAR
    Sieve Bootstrap 0.5444  0.5473 0.5544            ARMA


Archivo 'Base_Tamaño_3_escenarios.xlsx' creado exitosamente!
Columna 'Block Bootstrapping' verificada en el archivo final.
Total de filas: 25200


### Analisis general

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
from pathlib import Path

# ====================================================================================
# 1. CONFIGURACIÓN Y PREPARACIÓN DE DATOS
# ====================================================================================
warnings.filterwarnings('ignore')

# Configuración de gráficas
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 11
plt.rcParams['font.family'] = 'serif'

# Rutas
output_dir = Path("./Resultados_analisis/Tamaño")
output_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
try:
    df = pd.read_excel("./datos/Simulacion/Tamaño/Base_Tamaño_3_escenarios.xlsx")
except FileNotFoundError:
    print("Error: No se encontró el archivo Excel. Verifica la ruta.")
    exit()

# Renombrar escenarios
df['ESCENARIO'] = df['ESCENARIO'].replace({
    "Lineal Estacionario": "Lineal Estacionario (ARMA)",
    "Lineal No estacionario": "Lineal No Estacionario (ARIMA)",
    "No lineal Estacionario": "No lineal Estacionario (SETAR)"
})

# Identificación de variables y modelos
var_cols = ['Paso', 'Proceso', 'Tipo_Proceso', 'Distribución', 'Varianza', 
            'N_Train', 'N_Calib', 'N_Total', 'Valor_Observado', 'ESCENARIO', 'Size']

model_cols = [col for col in df.columns 
              if col not in var_cols and pd.api.types.is_numeric_dtype(df[col])]

n_total_values = sorted(df['N_Total'].unique())
escenarios = df['ESCENARIO'].unique()

# ====================================================================================
# PALETA DE COLORES ÚNICA Y DISTINTIVA PARA MODELOS
# ====================================================================================
def generate_maximally_distinct_colors(n):
    """
    Genera n colores MAXIMAMENTE distintivos usando diferentes estrategias
    combinadas para evitar colores similares
    """
    if n <= 10:
        base_colors = plt.cm.tab10(np.linspace(0, 1, 10))
        return base_colors[:n]
    elif n <= 20:
        base_colors = plt.cm.tab20(np.linspace(0, 1, 20))
        return base_colors[:n]
    else:
        colors = []
        
        # Estrategia 1: Colores base de tab20
        tab20 = plt.cm.tab20(np.linspace(0, 1, 20))
        colors.extend(tab20)
        
        # Estrategia 2: Colores de Set3 (pasteles)
        if n > 20:
            set3 = plt.cm.Set3(np.linspace(0, 1, 12))
            colors.extend(set3)
        
        # Estrategia 3: Colores de Paired
        if n > 32:
            paired = plt.cm.Paired(np.linspace(0, 1, 12))
            colors.extend(paired)
        
        # Estrategia 4: Generar colores en HSV con máxima separación
        if n > 44:
            remaining = n - len(colors)
            hsv_colors = []
            for i in range(remaining):
                hue = (i * 0.618033988749895) % 1.0  # Golden ratio
                saturation = 0.6 + (i % 3) * 0.15
                value = 0.7 + (i % 2) * 0.2
                rgb = plt.cm.hsv(hue)
                hsv_colors.append(rgb)
            colors.extend(hsv_colors)
        
        return np.array(colors[:n])

MODEL_COLORS = dict(zip(model_cols, generate_maximally_distinct_colors(len(model_cols))))

print(f"\n{'='*80}")
print(f"PALETA DE COLORES GENERADA PARA {len(model_cols)} MODELOS")
print(f"{'='*80}")
for i, (model, color) in enumerate(MODEL_COLORS.items(), 1):
    print(f"{i:2d}. {model:30s} -> RGB: ({color[0]:.3f}, {color[1]:.3f}, {color[2]:.3f})")
print(f"{'='*80}\n")

# ====================================================================================
# 2. TEST DIEBOLD-MARIANO MODIFICADO
# ====================================================================================

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """Test Diebold-Mariano con fixed-smoothing asymptotics."""
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    if T < 2: 
        return np.nan, np.nan, np.nan
    
    u = d - d_bar
    m = max(1, int(np.floor(T**(1/3))))
    
    from scipy.fft import fft
    fft_u = fft(u)
    periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
    
    if m >= len(periodogram) - 1: 
        m = len(periodogram) - 2
    
    sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
    if sigma_hat_sq <= 0: 
        sigma_hat_sq = np.var(d, ddof=1) / T
    
    dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
    df_t = 2 * m
    p_value = 2 * (1 - stats.t.cdf(abs(dm_stat), df_t))
    
    return dm_stat, p_value, d_bar

# ====================================================================================
# 2B. CORRECCIÓN DE BONFERRONI
# ====================================================================================

def bonferroni_correction(p_values, alpha=0.05):
    """
    Aplica corrección de Bonferroni a una lista de p-values.
    
    Parámetros:
    -----------
    p_values : array-like
        Lista de p-values a corregir
    alpha : float
        Nivel de significancia nominal (default: 0.05)
    
    Retorna:
    --------
    corrected_alpha : float
        Nivel de significancia ajustado (alpha / número de tests)
    significant_bonferroni : array-like
        Array booleano indicando si cada test es significativo tras corrección
    """
    n_comparisons = len(p_values)
    corrected_alpha = alpha / n_comparisons
    significant_bonferroni = np.array(p_values) < corrected_alpha
    
    return corrected_alpha, significant_bonferroni

def classify_significance(p_value, p_bonf, alpha=0.05, alpha_bonf=None):
    """
    Clasifica la significancia de un test considerando tanto el p-value original
    como la corrección de Bonferroni.
    
    Parámetros:
    -----------
    p_value : float
        P-value sin corregir
    p_bonf : float
        P-value con corrección de Bonferroni
    alpha : float
        Nivel de significancia nominal
    alpha_bonf : float
        Nivel de significancia corregido por Bonferroni
    
    Retorna:
    --------
    tuple : (clasificacion_sin_correccion, clasificacion_con_bonferroni)
    """
    # Sin corrección
    if p_value < 0.01:
        sin_corr = 'Sí (p<0.01)'
    elif p_value < 0.05:
        sin_corr = 'Sí (p<0.05)'
    elif p_value < 0.1:
        sin_corr = 'Marginal (p<0.1)'
    else:
        sin_corr = 'No'
    
    # Con corrección de Bonferroni
    if alpha_bonf is None:
        alpha_bonf = 0.05  # valor por defecto
    
    if p_bonf:  # Si pasó la corrección de Bonferroni
        if p_value < alpha_bonf:
            con_bonf = f'Sí (Bonferroni α={alpha_bonf:.6f})'
        else:
            con_bonf = 'No (post-Bonferroni)'
    else:
        con_bonf = 'No (post-Bonferroni)'
    
    return sin_corr, con_bonf

# ====================================================================================
# 3. HEATMAP: MODELOS VS N_TOTAL (Z-SCORES POR MODELO)
# ====================================================================================

def plot_heatmap_zscores_models_vs_ntotal(data, scenario_name=None):
    """
    Heatmap de Modelos (filas) vs N_Total (columnas) con Z-scores
    Z-score calculado POR MODELO (estandarización por fila)
    """
    if scenario_name:
        data = data[data['ESCENARIO'] == scenario_name]
        suffix = scenario_name.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Z-scores de ECRPS: Modelos vs N_Total\n{scenario_name}'
    else:
        suffix = "General"
        title = 'Z-scores de ECRPS: Modelos vs N_Total\nTodos los Escenarios'
    
    # Crear matriz de ECRPS promedio
    ecrps_data = []
    for model in model_cols:
        row = []
        for nt in n_total_values:
            ecrps_mean = data[data['N_Total'] == nt][model].mean()
            row.append(ecrps_mean)
        ecrps_data.append(row)
    
    ecrps_df = pd.DataFrame(ecrps_data, 
                             index=model_cols, 
                             columns=[f'N={nt}' for nt in n_total_values])
    
    # Calcular Z-scores POR MODELO (por fila)
    zscore_df = ecrps_df.apply(lambda row: (row - row.mean()) / row.std() if row.std() > 0 else 0, axis=1)
    
    # Guardar datos
    ecrps_df.to_excel(output_dir / f'heatmap_ecrps_{suffix}.xlsx')
    zscore_df.to_excel(output_dir / f'heatmap_zscores_{suffix}.xlsx')
    
    # Graficar
    plt.figure(figsize=(10, max(8, len(model_cols) * 0.4)))
    sns.heatmap(zscore_df, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, vmin=-2, vmax=2,
                cbar_kws={'label': 'Z-score'},
                linewidths=0.5, linecolor='gray')
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Tamaño Muestral Total', fontsize=12)
    plt.ylabel('Modelo', fontsize=12)
    plt.tight_layout()
    plt.savefig(output_dir / f'heatmap_zscore_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Heatmap Z-scores generado: {suffix}")

# ====================================================================================
# 4. TEST DM CON BONFERRONI: COMPARACIÓN ENTRE TODOS LOS TAMAÑOS (FORMATO EXCEL)
# ====================================================================================

def dm_test_all_sizes_excel(data, scenario_name=None, alpha=0.05):
    """
    Test DM comparando TODOS los pares de tamaños para cada modelo
    CON CORRECCIÓN DE BONFERRONI
    
    Formato: Modelo | Tamaño1 | Tamaño2 | ECRPS1 | ECRPS2 | Mejora% | 
             DM_Stat | p_value | Significativo | Significativo_Bonferroni
    """
    if scenario_name:
        data = data[data['ESCENARIO'] == scenario_name]
        suffix = scenario_name.replace(" ", "_").replace("(", "").replace(")", "")
    else:
        suffix = "General"
    
    results = []
    p_values_list = []
    
    # PASO 1: Realizar todos los tests y recopilar p-values
    for model in model_cols:
        for i, n1 in enumerate(n_total_values):
            for n2 in n_total_values[i+1:]:
                
                errors1 = data[data['N_Total'] == n1][model].values
                errors2 = data[data['N_Total'] == n2][model].values
                
                min_len = min(len(errors1), len(errors2))
                errors1 = errors1[:min_len]
                errors2 = errors2[:min_len]
                
                ecrps1 = errors1.mean()
                ecrps2 = errors2.mean()
                
                mejora_absoluta = ecrps1 - ecrps2
                mejora_relativa = (mejora_absoluta / ecrps1) * 100 if ecrps1 != 0 else 0
                
                dm_stat, p_value, _ = modified_diebold_mariano_test(errors1, errors2)
                
                results.append({
                    'Modelo': model,
                    'Tamaño_1': n1,
                    'Tamaño_2': n2,
                    'ECRPS_Promedio_1': ecrps1,
                    'ECRPS_Promedio_2': ecrps2,
                    'Mejora_%': mejora_relativa,
                    'DM_Statistic': dm_stat,
                    'p_value': p_value
                })
                
                p_values_list.append(p_value)
    
    # PASO 2: Aplicar corrección de Bonferroni
    n_comparisons = len(p_values_list)
    alpha_bonferroni = alpha / n_comparisons
    
    print(f"\n{'='*80}")
    print(f"CORRECCIÓN DE BONFERRONI - {suffix}")
    print(f"{'='*80}")
    print(f"Número total de comparaciones: {n_comparisons}")
    print(f"α nominal: {alpha}")
    print(f"α corregido (Bonferroni): {alpha_bonferroni:.6f}")
    print(f"{'='*80}\n")
    
    # PASO 3: Clasificar significancia con y sin corrección
    for i, result in enumerate(results):
        p_val = result['p_value']
        
        # Significancia sin corrección
        if p_val < 0.01:
            sig_sin_corr = 'Sí (p<0.01)'
        elif p_val < 0.05:
            sig_sin_corr = 'Sí (p<0.05)'
        elif p_val < 0.1:
            sig_sin_corr = 'Marginal (p<0.1)'
        else:
            sig_sin_corr = 'No'
        
        # Significancia con Bonferroni
        if p_val < alpha_bonferroni:
            sig_bonf = f'Sí (p<{alpha_bonferroni:.6f})'
        else:
            sig_bonf = 'No'
        
        result['Significativo_Sin_Corrección'] = sig_sin_corr
        result['Significativo_Bonferroni'] = sig_bonf
        result['Alpha_Bonferroni'] = alpha_bonferroni
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(['Modelo', 'Tamaño_1', 'Tamaño_2'])
    
    # PASO 4: Guardar en Excel con formato mejorado
    excel_path = output_dir / f'dm_bonferroni_comparacion_{suffix}.xlsx'
    with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
        results_df.to_excel(writer, sheet_name='Comparaciones_DM_Bonferroni', index=False)
        
        workbook = writer.book
        worksheet = writer.sheets['Comparaciones_DM_Bonferroni']
        
        # Formatos
        format_header = workbook.add_format({
            'bold': True,
            'bg_color': '#4CAF50',
            'font_color': 'white',
            'align': 'center',
            'valign': 'vcenter',
            'border': 1
        })
        
        format_number = workbook.add_format({'num_format': '0.0000', 'align': 'center'})
        format_percent = workbook.add_format({'num_format': '0.00%', 'align': 'center'})
        format_pvalue = workbook.add_format({'num_format': '0.000000', 'align': 'center'})
        format_sig_yes = workbook.add_format({
            'bg_color': '#90EE90',
            'align': 'center',
            'border': 1
        })
        format_sig_no = workbook.add_format({
            'bg_color': '#FFB6C1',
            'align': 'center',
            'border': 1
        })
        
        # Aplicar formato a encabezados
        for col_num, value in enumerate(results_df.columns.values):
            worksheet.write(0, col_num, value, format_header)
        
        # Ajustar anchos de columna
        worksheet.set_column('A:A', 30)  # Modelo
        worksheet.set_column('B:C', 12)  # Tamaños
        worksheet.set_column('D:E', 15)  # ECRPS
        worksheet.set_column('F:F', 12)  # Mejora%
        worksheet.set_column('G:G', 13)  # DM Stat
        worksheet.set_column('H:H', 12)  # p-value
        worksheet.set_column('I:I', 22)  # Sig sin corrección
        worksheet.set_column('J:J', 22)  # Sig con Bonferroni
        worksheet.set_column('K:K', 18)  # Alpha Bonferroni
        
        # Aplicar formatos
        for row_num in range(1, len(results_df) + 1):
            worksheet.write_number(row_num, 3, results_df.iloc[row_num-1]['ECRPS_Promedio_1'], format_number)
            worksheet.write_number(row_num, 4, results_df.iloc[row_num-1]['ECRPS_Promedio_2'], format_number)
            worksheet.write_number(row_num, 5, results_df.iloc[row_num-1]['Mejora_%'] / 100, format_percent)
            worksheet.write_number(row_num, 6, results_df.iloc[row_num-1]['DM_Statistic'], format_number)
            worksheet.write_number(row_num, 7, results_df.iloc[row_num-1]['p_value'], format_pvalue)
            worksheet.write_number(row_num, 10, alpha_bonferroni, format_pvalue)
            
            # Formato condicional para significancia Bonferroni
            sig_bonf = results_df.iloc[row_num-1]['Significativo_Bonferroni']
            if sig_bonf.startswith('Sí'):
                worksheet.write(row_num, 9, sig_bonf, format_sig_yes)
            else:
                worksheet.write(row_num, 9, sig_bonf, format_sig_no)
        
        # Agregar hoja de resumen
        summary_data = {
            'Métrica': [
                'Total de comparaciones',
                'α nominal',
                'α corregido (Bonferroni)',
                'Significativas sin corrección (p<0.05)',
                'Significativas con Bonferroni',
                '% Supervivencia post-Bonferroni'
            ],
            'Valor': [
                n_comparisons,
                alpha,
                alpha_bonferroni,
                len(results_df[results_df['p_value'] < 0.05]),
                len(results_df[results_df['p_value'] < alpha_bonferroni]),
                f"{(len(results_df[results_df['p_value'] < alpha_bonferroni]) / len(results_df[results_df['p_value'] < 0.05]) * 100) if len(results_df[results_df['p_value'] < 0.05]) > 0 else 0:.2f}%"
            ]
        }
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Resumen_Bonferroni', index=False)
        
        worksheet_summary = writer.sheets['Resumen_Bonferroni']
        for col_num, value in enumerate(summary_df.columns.values):
            worksheet_summary.write(0, col_num, value, format_header)
        worksheet_summary.set_column('A:A', 40)
        worksheet_summary.set_column('B:B', 25)
    
    print(f"✓ Test DM con Bonferroni (Excel) generado: {suffix}")
    print(f"  - {n_comparisons} comparaciones totales")
    print(f"  - {len(results_df[results_df['p_value'] < 0.05])} significativas sin corrección (p<0.05)")
    print(f"  - {len(results_df[results_df['p_value'] < alpha_bonferroni])} significativas con Bonferroni\n")
    
    return results_df

# ====================================================================================
# 5. GRÁFICA DE MEJORA RELATIVA
# ====================================================================================

def plot_relative_improvement(data, scenario_name=None):
    """
    Mejora relativa (%) respecto al tamaño base (primer N_Total)
    """
    if scenario_name:
        data = data[data['ESCENARIO'] == scenario_name]
        suffix = scenario_name.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Mejora Relativa vs N={n_total_values[0]} - {scenario_name}'
    else:
        suffix = "General"
        title = f'Mejora Relativa vs N={n_total_values[0]} - Todos los Escenarios'
    
    baseline = n_total_values[0]
    improvements = {}
    
    for model in model_cols:
        base_perf = data[data['N_Total'] == baseline][model].mean()
        model_impr = []
        for nt in n_total_values:
            current = data[data['N_Total'] == nt][model].mean()
            improvement = ((base_perf - current) / base_perf) * 100 if base_perf != 0 else 0
            model_impr.append(improvement)
        improvements[model] = model_impr
    
    # Guardar datos
    improvements_df = pd.DataFrame(improvements, index=n_total_values)
    improvements_df.index.name = 'N_Total'
    improvements_df.to_excel(output_dir / f'mejora_relativa_{suffix}.xlsx')
    
    # Graficar
    plt.figure(figsize=(12, 7))
    for model, values in improvements.items():
        plt.plot(n_total_values, values, marker='o', label=model, 
                color=MODEL_COLORS[model], linewidth=2.5, markersize=8)
    
    plt.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('N_Total', fontsize=12)
    plt.ylabel('Mejora Relativa (%)', fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    plt.grid(True, alpha=0.3, linestyle=':', linewidth=0.8)
    plt.tight_layout()
    plt.savefig(output_dir / f'mejora_relativa_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Mejora relativa generada: {suffix}")

# ====================================================================================
# 6. EJECUCIÓN PRINCIPAL
# ====================================================================================

print("\n" + "="*80)
print("SIMULACIÓN 3: EFECTOS DEL TAMAÑO MUESTRAL ABSOLUTO")
print("CON TEST DIEBOLD-MARIANO MODIFICADO Y CORRECCIÓN DE BONFERRONI")
print("="*80 + "\n")

print(f"Modelos evaluados: {len(model_cols)}")
print(f"Tamaños muestrales: {n_total_values}")
print(f"Escenarios: {list(escenarios)}\n")

# 1. HEATMAPS
print("="*80)
print("1. GENERANDO HEATMAPS: Z-scores por Modelo")
print("="*80)
plot_heatmap_zscores_models_vs_ntotal(df)
for scen in escenarios:
    plot_heatmap_zscores_models_vs_ntotal(df, scen)
print()

# 2. TEST DM CON BONFERRONI
print("="*80)
print("2. TEST DM CON CORRECCIÓN DE BONFERRONI")
print("="*80)
dm_results_general = dm_test_all_sizes_excel(df)
for scen in escenarios:
    dm_test_all_sizes_excel(df, scen)
print()

# 3. MEJORA RELATIVA
print("="*80)
print("3. GRÁFICAS DE MEJORA RELATIVA")
print("="*80)
plot_relative_improvement(df)
for scen in escenarios:
    plot_relative_improvement(df, scen)
print()

# ====================================================================================
# 7. RESUMEN EJECUTIVO CON BONFERRONI
# ====================================================================================

print("="*80)
print("RESUMEN EJECUTIVO")
print("="*80)

# Mejor modelo por tamaño
print("\nMejor modelo por tamaño muestral (menor ECRPS promedio):")
print("-" * 60)
for nt in n_total_values:
    data_nt = df[df['N_Total'] == nt]
    best_model = None
    best_ecrps = float('inf')
    for model in model_cols:
        ecrps = data_nt[model].mean()
        if ecrps < best_ecrps:
            best_ecrps = ecrps
            best_model = model
    print(f"  N={nt:4d}: {best_model:30s} (ECRPS = {best_ecrps:.4f})")

# Análisis de mejoras significativas con Bonferroni
print(f"\nComparaciones N={n_total_values[0]} vs N={n_total_values[-1]} (CON BONFERRONI):")
print("-" * 60)
comparacion_extremos = dm_results_general[
    (dm_results_general['Tamaño_1'] == n_total_values[0]) & 
    (dm_results_general['Tamaño_2'] == n_total_values[-1])
]
comparacion_extremos = comparacion_extremos.sort_values('Mejora_%', ascending=False)

# Significativas con Bonferroni
sig_bonf = comparacion_extremos[comparacion_extremos['Significativo_Bonferroni'].str.startswith('Sí')]
print(f"\nModelos con mejoras significativas (post-Bonferroni): {len(sig_bonf)}")
if len(sig_bonf) > 0:
    print("Top 5 mejoras significativas (Bonferroni):")
    for _, row in sig_bonf.head(5).iterrows():
        print(f"  {row['Modelo']:30s}: {row['Mejora_%']:6.2f}% - {row['Significativo_Bonferroni']}")
else:
    print("  No hay mejoras significativas tras corrección de Bonferroni")

print("\nTop 5 mayores mejoras (sin considerar Bonferroni):")
for _, row in comparacion_extremos.head(5).iterrows():
    print(f"  {row['Modelo']:30s}: {row['Mejora_%']:6.2f}% - Sin corr: {row['Significativo_Sin_Corrección']} | Bonf: {row['Significativo_Bonferroni']}")

print("\nMenores mejoras o empeoramientos:")
for _, row in comparacion_extremos.tail(5).iterrows():
    print(f"  {row['Modelo']:30s}: {row['Mejora_%']:6.2f}% - Sin corr: {row['Significativo_Sin_Corrección']} | Bonf: {row['Significativo_Bonferroni']}")

print("\n" + "="*80)
print("ANÁLISIS COMPLETADO EXITOSAMENTE")
print("="*80)
print(f"Resultados guardados en: {output_dir}")
print(f"\nArchivos generados:")
print(f"  - 4 Heatmaps con Z-scores (1 general + 3 por escenario)")
print(f"  - 4 Archivos Excel con Test DM y Bonferroni (1 general + 3 por escenario)")
print(f"  - 4 Gráficas de mejora relativa (1 general + 3 por escenario)")
print(f"  - Cada Excel incluye hoja de resumen con estadísticas de Bonferroni")
print("="*80 + "\n")


PALETA DE COLORES GENERADA PARA 9 MODELOS
 1. Block Bootstrapping            -> RGB: (0.122, 0.467, 0.706)
 2. Sieve Bootstrap                -> RGB: (1.000, 0.498, 0.055)
 3. LSPM                           -> RGB: (0.173, 0.627, 0.173)
 4. LSPMW                          -> RGB: (0.839, 0.153, 0.157)
 5. AREPD                          -> RGB: (0.580, 0.404, 0.741)
 6. MCPS                           -> RGB: (0.549, 0.337, 0.294)
 7. AV-MCPS                        -> RGB: (0.890, 0.467, 0.761)
 8. DeepAR                         -> RGB: (0.498, 0.498, 0.498)
 9. EnCQR-LSTM                     -> RGB: (0.737, 0.741, 0.133)


SIMULACIÓN 3: EFECTOS DEL TAMAÑO MUESTRAL ABSOLUTO
CON TEST DIEBOLD-MARIANO MODIFICADO Y CORRECCIÓN DE BONFERRONI

Modelos evaluados: 9
Tamaños muestrales: [np.int64(120), np.int64(240), np.int64(360), np.int64(600), np.int64(1200)]
Escenarios: ['Lineal Estacionario (ARMA)', 'Lineal No Estacionario (ARIMA)', 'No lineal Estacionario (SETAR)']

1. GENERANDO HEATMAPS: Z-

## Analisis proporciones

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
from pathlib import Path

# ====================================================================================
# 1. CONFIGURACIÓN Y PREPARACIÓN DE DATOS
# ====================================================================================
warnings.filterwarnings('ignore')

# Configuración de gráficas
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 11
plt.rcParams['font.family'] = 'serif'

# Rutas
output_dir = Path("./Resultados_analisis/Proporciones")
output_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
try:
    df = pd.read_excel("./datos/Simulacion/proporciones/resultados_PROPORCIONES_240_TODOS.xlsx")
except FileNotFoundError:
    print("Error: No se encontró el archivo Excel. Verifica la ruta.")
    exit()

# Renombrar escenarios
df['Tipo_Proceso'] = df['Tipo_Proceso'].replace({
    "ARMA": "Lineal Estacionario (ARMA)",
    "ARIMA": "Lineal No Estacionario (ARIMA)",
    "SETAR": "No Lineal Estacionario (SETAR)"
})

# Limpiar Prop_Calib
def clean_prop_calib(value):
    if pd.isna(value):
        return np.nan
    value_str = str(value).strip()
    if '%' in value_str:
        value_str = value_str.split('%')[0]
    try:
        value_num = float(value_str)
        if value_num < 1:
            return int(value_num * 100)
        else:
            return int(value_num)
    except:
        return np.nan

df['Prop_Calib_Pct'] = df['Prop_Calib'].apply(clean_prop_calib)
df['Prop_Calib'] = df['Prop_Calib_Pct'] / 100

# Limpiar Paso
def clean_paso(value):
    if pd.isna(value):
        return np.nan
    value_str = str(value).strip()
    import re
    numbers = re.findall(r'\d+', value_str)
    if numbers:
        return int(numbers[0])
    else:
        return np.nan

df['Paso'] = df['Paso'].apply(clean_paso)

# Identificación de variables y modelos
var_cols = ['Paso', 'Proceso', 'Distribución', 'Varianza', 
            'N_Train', 'N_Calib', 'Prop_Calib', 'Prop_Calib_Pct', 'Tipo_Proceso']
model_cols = [col for col in df.columns if col not in var_cols and pd.api.types.is_numeric_dtype(df[col])]

# Limpiar datos numéricos
for col in ['N_Train', 'N_Calib', 'Varianza', 'Paso']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

for model in model_cols:
    df[model] = pd.to_numeric(df[model], errors='coerce')

# Eliminar NaN
critical_cols = ['Paso', 'Prop_Calib_Pct', 'N_Train', 'N_Calib'] + model_cols
df = df.dropna(subset=critical_cols)

prop_values = sorted(df['Prop_Calib_Pct'].unique())
escenarios = df['Tipo_Proceso'].unique()

# ====================================================================================
# PALETA DE COLORES ÚNICA PARA MODELOS
# ====================================================================================
def generate_maximally_distinct_colors(n):
    if n <= 10:
        base_colors = plt.cm.tab10(np.linspace(0, 1, 10))
        return base_colors[:n]
    elif n <= 20:
        base_colors = plt.cm.tab20(np.linspace(0, 1, 20))
        return base_colors[:n]
    else:
        colors = []
        tab20 = plt.cm.tab20(np.linspace(0, 1, 20))
        colors.extend(tab20)
        
        if n > 20:
            set3 = plt.cm.Set3(np.linspace(0, 1, 12))
            colors.extend(set3)
        
        if n > 32:
            paired = plt.cm.Paired(np.linspace(0, 1, 12))
            colors.extend(paired)
        
        if n > 44:
            remaining = n - len(colors)
            hsv_colors = []
            for i in range(remaining):
                hue = (i * 0.618033988749895) % 1.0
                saturation = 0.6 + (i % 3) * 0.15
                value = 0.7 + (i % 2) * 0.2
                rgb = plt.cm.hsv(hue)
                hsv_colors.append(rgb)
            colors.extend(hsv_colors)
        
        return np.array(colors[:n])

MODEL_COLORS = dict(zip(model_cols, generate_maximally_distinct_colors(len(model_cols))))

print(f"\n{'='*80}")
print(f"SIMULACIÓN 4: PROPORCIONES DE CALIBRACIÓN (N=240 FIJO)")
print(f"CON TEST DIEBOLD-MARIANO MODIFICADO Y CORRECCIÓN DE BONFERRONI")
print(f"{'='*80}")
print(f"\nModelos evaluados: {len(model_cols)}")
print(f"Proporciones: {prop_values}")
print(f"Escenarios: {list(escenarios)}\n")

# ====================================================================================
# 2. TEST DIEBOLD-MARIANO MODIFICADO
# ====================================================================================

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """Test Diebold-Mariano con fixed-smoothing asymptotics."""
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    if T < 2: 
        return np.nan, np.nan, np.nan
    
    u = d - d_bar
    m = max(1, int(np.floor(T**(1/3))))
    
    from scipy.fft import fft
    fft_u = fft(u)
    periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
    
    if m >= len(periodogram) - 1: 
        m = len(periodogram) - 2
    
    sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
    if sigma_hat_sq <= 0: 
        sigma_hat_sq = np.var(d, ddof=1) / T
    
    dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
    df_t = 2 * m
    p_value = 2 * (1 - stats.t.cdf(abs(dm_stat), df_t))
    
    return dm_stat, p_value, d_bar

# ====================================================================================
# 3. HEATMAP: MODELOS VS PROPORCIONES (Z-SCORES)
# ====================================================================================

def plot_heatmap_zscores_models_vs_props(data, scenario_name=None):
    """Heatmap de Modelos vs Proporciones con Z-scores por modelo"""
    if scenario_name:
        data = data[data['Tipo_Proceso'] == scenario_name]
        suffix = scenario_name.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Z-scores de ECRPS: Modelos vs Proporción\n{scenario_name}'
    else:
        suffix = "General"
        title = 'Z-scores de ECRPS: Modelos vs Proporción\nTodos los Escenarios'
    
    # Crear matriz de ECRPS promedio
    ecrps_data = []
    for model in model_cols:
        row = []
        for prop in prop_values:
            ecrps_mean = data[data['Prop_Calib_Pct'] == prop][model].mean()
            row.append(ecrps_mean)
        ecrps_data.append(row)
    
    ecrps_df = pd.DataFrame(ecrps_data, 
                             index=model_cols, 
                             columns=[f'{p}%' for p in prop_values])
    
    # Calcular Z-scores POR MODELO (por fila)
    zscore_df = ecrps_df.apply(lambda row: (row - row.mean()) / row.std() if row.std() > 0 else 0, axis=1)
    
    # Guardar datos
    ecrps_df.to_excel(output_dir / f'heatmap_ecrps_{suffix}.xlsx')
    zscore_df.to_excel(output_dir / f'heatmap_zscores_{suffix}.xlsx')
    
    # Graficar
    plt.figure(figsize=(10, max(8, len(model_cols) * 0.4)))
    sns.heatmap(zscore_df, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, vmin=-2, vmax=2,
                cbar_kws={'label': 'Z-score'},
                linewidths=0.5, linecolor='gray')
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Proporción de Calibración', fontsize=12)
    plt.ylabel('Modelo', fontsize=12)
    plt.tight_layout()
    plt.savefig(output_dir / f'heatmap_zscore_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Heatmap Z-scores generado: {suffix}")

# ====================================================================================
# 4. TEST DM CON BONFERRONI: COMPARACIÓN ENTRE PROPORCIONES (FORMATO EXCEL)
# ====================================================================================

def dm_test_all_props_excel(data, scenario_name=None, alpha=0.05):
    """
    Test DM comparando todos los pares de proporciones para cada modelo
    CON CORRECCIÓN DE BONFERRONI
    """
    if scenario_name:
        data = data[data['Tipo_Proceso'] == scenario_name]
        suffix = scenario_name.replace(" ", "_").replace("(", "").replace(")", "")
    else:
        suffix = "General"
    
    results = []
    p_values_list = []
    
    # Calcular número de comparaciones
    n_props = len(prop_values)
    n_comparisons_per_model = (n_props * (n_props - 1)) // 2
    n_total_comparisons = len(model_cols) * n_comparisons_per_model
    
    # PASO 1: Realizar todos los tests y recopilar p-values
    for model in model_cols:
        for i, prop1 in enumerate(prop_values):
            for prop2 in prop_values[i+1:]:
                
                errors1 = data[data['Prop_Calib_Pct'] == prop1][model].values
                errors2 = data[data['Prop_Calib_Pct'] == prop2][model].values
                
                min_len = min(len(errors1), len(errors2))
                errors1 = errors1[:min_len]
                errors2 = errors2[:min_len]
                
                ecrps1 = errors1.mean()
                ecrps2 = errors2.mean()
                
                mejora_absoluta = ecrps1 - ecrps2
                mejora_relativa = (mejora_absoluta / ecrps1) * 100 if ecrps1 != 0 else 0
                
                dm_stat, p_value, _ = modified_diebold_mariano_test(errors1, errors2)
                
                results.append({
                    'Modelo': model,
                    'Proporcion_1_%': prop1,
                    'Proporcion_2_%': prop2,
                    'ECRPS_Promedio_1': ecrps1,
                    'ECRPS_Promedio_2': ecrps2,
                    'Mejora_%': mejora_relativa,
                    'DM_Statistic': dm_stat,
                    'p_value': p_value
                })
                
                p_values_list.append(p_value)
    
    # PASO 2: Aplicar corrección de Bonferroni
    alpha_bonferroni = alpha / n_total_comparisons
    
    print(f"\n{'='*80}")
    print(f"CORRECCIÓN DE BONFERRONI - {suffix}")
    print(f"{'='*80}")
    print(f"Número de modelos: {len(model_cols)}")
    print(f"Comparaciones por modelo: {n_comparisons_per_model}")
    print(f"Total de comparaciones: {n_total_comparisons}")
    print(f"α nominal: {alpha}")
    print(f"α corregido (Bonferroni): {alpha_bonferroni:.8f}")
    print(f"{'='*80}\n")
    
    # PASO 3: Clasificar significancia con y sin corrección
    for i, result in enumerate(results):
        p_val = result['p_value']
        
        # Significancia sin corrección
        if p_val < 0.01:
            sig_sin_corr = 'Sí (p<0.01)'
        elif p_val < 0.05:
            sig_sin_corr = 'Sí (p<0.05)'
        elif p_val < 0.1:
            sig_sin_corr = 'Marginal (p<0.1)'
        else:
            sig_sin_corr = 'No'
        
        # Significancia con Bonferroni
        if p_val < alpha_bonferroni:
            sig_bonf = f'Sí (p<{alpha_bonferroni:.8f})'
        else:
            sig_bonf = 'No'
        
        result['Significativo_Sin_Corrección'] = sig_sin_corr
        result['Significativo_Bonferroni'] = sig_bonf
        result['Alpha_Bonferroni'] = alpha_bonferroni
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(['Modelo', 'Proporcion_1_%', 'Proporcion_2_%'])
    
    # PASO 4: Guardar en Excel con formato mejorado
    excel_path = output_dir / f'dm_bonferroni_proporciones_{suffix}.xlsx'
    with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
        results_df.to_excel(writer, sheet_name='Comparaciones_DM_Bonferroni', index=False)
        
        workbook = writer.book
        worksheet = writer.sheets['Comparaciones_DM_Bonferroni']
        
        # Formatos
        format_header = workbook.add_format({
            'bold': True,
            'bg_color': '#4CAF50',
            'font_color': 'white',
            'align': 'center',
            'valign': 'vcenter',
            'border': 1
        })
        
        format_number = workbook.add_format({'num_format': '0.0000', 'align': 'center'})
        format_percent = workbook.add_format({'num_format': '0.00%', 'align': 'center'})
        format_pvalue = workbook.add_format({'num_format': '0.00000000', 'align': 'center'})
        format_sig_yes = workbook.add_format({
            'bg_color': '#90EE90',
            'align': 'center',
            'border': 1
        })
        format_sig_no = workbook.add_format({
            'bg_color': '#FFB6C1',
            'align': 'center',
            'border': 1
        })
        
        # Aplicar formato a encabezados
        for col_num, value in enumerate(results_df.columns.values):
            worksheet.write(0, col_num, value, format_header)
        
        # Ajustar anchos de columna
        worksheet.set_column('A:A', 30)  # Modelo
        worksheet.set_column('B:C', 15)  # Proporciones
        worksheet.set_column('D:E', 18)  # ECRPS
        worksheet.set_column('F:F', 12)  # Mejora%
        worksheet.set_column('G:G', 13)  # DM Stat
        worksheet.set_column('H:H', 14)  # p-value
        worksheet.set_column('I:I', 22)  # Sig sin corrección
        worksheet.set_column('J:J', 28)  # Sig con Bonferroni
        worksheet.set_column('K:K', 18)  # Alpha Bonferroni
        
        # Aplicar formatos
        for row_num in range(1, len(results_df) + 1):
            worksheet.write_number(row_num, 3, results_df.iloc[row_num-1]['ECRPS_Promedio_1'], format_number)
            worksheet.write_number(row_num, 4, results_df.iloc[row_num-1]['ECRPS_Promedio_2'], format_number)
            worksheet.write_number(row_num, 5, results_df.iloc[row_num-1]['Mejora_%'] / 100, format_percent)
            worksheet.write_number(row_num, 6, results_df.iloc[row_num-1]['DM_Statistic'], format_number)
            worksheet.write_number(row_num, 7, results_df.iloc[row_num-1]['p_value'], format_pvalue)
            worksheet.write_number(row_num, 10, alpha_bonferroni, format_pvalue)
            
            # Formato condicional para significancia Bonferroni
            sig_bonf = results_df.iloc[row_num-1]['Significativo_Bonferroni']
            if sig_bonf.startswith('Sí'):
                worksheet.write(row_num, 9, sig_bonf, format_sig_yes)
            else:
                worksheet.write(row_num, 9, sig_bonf, format_sig_no)
        
        # Agregar hoja de resumen
        sig_sin_corr = len(results_df[results_df['p_value'] < 0.05])
        sig_con_bonf = len(results_df[results_df['p_value'] < alpha_bonferroni])
        tasa_supervivencia = (sig_con_bonf / sig_sin_corr * 100) if sig_sin_corr > 0 else 0
        
        summary_data = {
            'Métrica': [
                'Número de modelos',
                'Comparaciones por modelo',
                'Total de comparaciones',
                'α nominal',
                'α corregido (Bonferroni)',
                'Significativas sin corrección (p<0.05)',
                'Significativas con Bonferroni',
                '% Supervivencia post-Bonferroni'
            ],
            'Valor': [
                len(model_cols),
                n_comparisons_per_model,
                n_total_comparisons,
                alpha,
                alpha_bonferroni,
                sig_sin_corr,
                sig_con_bonf,
                f"{tasa_supervivencia:.2f}%"
            ]
        }
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Resumen_Bonferroni', index=False)
        
        worksheet_summary = writer.sheets['Resumen_Bonferroni']
        for col_num, value in enumerate(summary_df.columns.values):
            worksheet_summary.write(0, col_num, value, format_header)
        worksheet_summary.set_column('A:A', 45)
        worksheet_summary.set_column('B:B', 25)
    
    print(f"✓ Test DM con Bonferroni (Excel) generado: {suffix}")
    print(f"  - {n_total_comparisons} comparaciones totales")
    print(f"  - {sig_sin_corr} significativas sin corrección (p<0.05)")
    print(f"  - {sig_con_bonf} significativas con Bonferroni")
    print(f"  - {tasa_supervivencia:.2f}% tasa de supervivencia\n")
    
    return results_df

# ====================================================================================
# 5. GRÁFICA DE EVOLUCIÓN POR PROPORCIÓN
# ====================================================================================

def plot_evolution_by_proportion(data, scenario_name=None):
    """Evolución del ECRPS de cada modelo a través de proporciones"""
    if scenario_name:
        data = data[data['Tipo_Proceso'] == scenario_name]
        suffix = scenario_name.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Evolución del ECRPS por Proporción\n{scenario_name}'
    else:
        suffix = "General"
        title = 'Evolución del ECRPS por Proporción\nTodos los Escenarios'
    
    plt.figure(figsize=(12, 7))
    for model in model_cols:
        means = [data[data['Prop_Calib_Pct'] == p][model].mean() for p in prop_values]
        plt.plot(prop_values, means, marker='o', label=model, 
                color=MODEL_COLORS[model], linewidth=2.5, markersize=8)
    
    plt.xlabel('Proporción de Calibración (%)', fontsize=12)
    plt.ylabel('ECRPS Promedio', fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    plt.grid(True, alpha=0.3, linestyle=':', linewidth=0.8)
    plt.tight_layout()
    plt.savefig(output_dir / f'evolucion_proporciones_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Evolución por proporción generada: {suffix}")

# ====================================================================================
# 6. PROPORCIÓN ÓPTIMA POR MODELO
# ====================================================================================

def analyze_optimal_proportion(data, scenario_name=None):
    """Identifica la proporción óptima para cada modelo"""
    if scenario_name:
        data = data[data['Tipo_Proceso'] == scenario_name]
        suffix = scenario_name.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Proporción Óptima por Modelo\n{scenario_name}'
    else:
        suffix = "General"
        title = 'Proporción Óptima por Modelo\nTodos los Escenarios'
    
    optimal_results = []
    
    for model in model_cols:
        prop_means = {}
        for prop in prop_values:
            prop_means[prop] = data[data['Prop_Calib_Pct'] == prop][model].mean()
        
        optimal_prop = min(prop_means, key=prop_means.get)
        worst_prop = max(prop_means, key=prop_means.get)
        
        optimal_results.append({
            'Modelo': model,
            'Proporcion_Optima_%': optimal_prop,
            'ECRPS_Optimo': prop_means[optimal_prop],
            'Proporcion_Peor_%': worst_prop,
            'ECRPS_Peor': prop_means[worst_prop],
            'Diferencia_%': ((prop_means[worst_prop] - prop_means[optimal_prop]) / prop_means[optimal_prop]) * 100
        })
    
    optimal_df = pd.DataFrame(optimal_results).sort_values('ECRPS_Optimo')
    
    # Gráfica
    fig, ax = plt.subplots(figsize=(12, 8))
    
    y_pos = np.arange(len(optimal_df))
    colors = [MODEL_COLORS[m] for m in optimal_df['Modelo']]
    
    bars = ax.barh(y_pos, optimal_df['ECRPS_Optimo'], 
                   color=colors, alpha=0.85, edgecolor='black', linewidth=1.5)
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(optimal_df['Modelo'])
    ax.set_xlabel('ECRPS en Proporción Óptima', fontsize=11, fontweight='bold')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Anotar proporción óptima
    for i, (bar, (_, row)) in enumerate(zip(bars, optimal_df.iterrows())):
        width = bar.get_width()
        ax.text(width + width*0.01, bar.get_y() + bar.get_height()/2.,
               f"{row['Proporcion_Optima_%']}%",
               ha='left', va='center', fontsize=9, fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.5))
    
    plt.tight_layout()
    plt.savefig(output_dir / f'proporcion_optima_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Guardar tabla
    optimal_df.to_excel(output_dir / f'proporcion_optima_{suffix}.xlsx', index=False)
    
    print(f"✓ Análisis proporción óptima generado: {suffix}")
    
    return optimal_df

# ====================================================================================
# 7. ANÁLISIS DE SENSIBILIDAD A LA PROPORCIÓN
# ====================================================================================

def analyze_sensitivity_to_proportion(data, scenario_name=None):
    """Identifica qué modelos son más sensibles a cambios en la proporción"""
    if scenario_name:
        data = data[data['Tipo_Proceso'] == scenario_name]
        suffix = scenario_name.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Sensibilidad a Proporción de Calibración\n{scenario_name}'
    else:
        suffix = "General"
        title = 'Sensibilidad a Proporción de Calibración\nTodos los Escenarios'
    
    sensitivity_results = []
    
    for model in model_cols:
        means_by_prop = []
        for prop in prop_values:
            means_by_prop.append(data[data['Prop_Calib_Pct'] == prop][model].mean())
        
        means_array = np.array(means_by_prop)
        
        # Métricas de sensibilidad
        rango_absoluto = means_array.max() - means_array.min()
        rango_relativo = (rango_absoluto / means_array[0]) * 100
        volatilidad = np.std(means_array)
        cv = volatilidad / np.mean(means_array)
        
        # Correlación con proporción
        from scipy.stats import spearmanr
        corr, p_val = spearmanr(prop_values, means_array)
        
        sensitivity_results.append({
            'Modelo': model,
            'Rango_Absoluto': rango_absoluto,
            'Rango_Relativo_%': rango_relativo,
            'Volatilidad': volatilidad,
            'CV': cv,
            'Correlacion_Spearman': corr,
            'P_value': p_val,
            'ECRPS_Medio': means_array.mean()
        })
    
    sensitivity_df = pd.DataFrame(sensitivity_results).sort_values('Rango_Relativo_%', ascending=False)
    
    # Gráfica
    fig, ax = plt.subplots(figsize=(12, 8))
    
    colors = plt.cm.RdYlGn_r(sensitivity_df['Rango_Relativo_%'] / sensitivity_df['Rango_Relativo_%'].max())
    bars = ax.barh(sensitivity_df['Modelo'], sensitivity_df['Rango_Relativo_%'],
                  color=colors, alpha=0.85, edgecolor='black', linewidth=1.5)
    
    ax.set_xlabel('Variación Relativa Máxima (%)', fontsize=11, fontweight='bold')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    for bar, (_, row) in zip(bars, sensitivity_df.iterrows()):
        width = bar.get_width()
        ax.text(width + 0.5, bar.get_y() + bar.get_height()/2.,
               f"{row['Rango_Relativo_%']:.2f}%",
               ha='left', va='center', fontsize=8, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(output_dir / f'sensibilidad_proporcion_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    sensitivity_df.to_excel(output_dir / f'sensibilidad_proporcion_{suffix}.xlsx', index=False)
    
    print(f"✓ Análisis de sensibilidad generado: {suffix}")
    
    return sensitivity_df

# ====================================================================================
# 8. EJECUCIÓN PRINCIPAL
# ====================================================================================

print("="*80)
print("1. GENERANDO HEATMAPS: Z-scores por Modelo")
print("="*80)
plot_heatmap_zscores_models_vs_props(df)
for scen in escenarios:
    plot_heatmap_zscores_models_vs_props(df, scen)
print()

print("="*80)
print("2. TEST DM CON CORRECCIÓN DE BONFERRONI")
print("="*80)
dm_results_general = dm_test_all_props_excel(df)
for scen in escenarios:
    dm_test_all_props_excel(df, scen)
print()

print("="*80)
print("3. GRÁFICAS DE EVOLUCIÓN POR PROPORCIÓN")
print("="*80)
plot_evolution_by_proportion(df)
for scen in escenarios:
    plot_evolution_by_proportion(df, scen)
print()

print("="*80)
print("4. ANÁLISIS DE PROPORCIÓN ÓPTIMA")
print("="*80)
optimal_general = analyze_optimal_proportion(df)
for scen in escenarios:
    analyze_optimal_proportion(df, scen)
print()

print("="*80)
print("5. ANÁLISIS DE SENSIBILIDAD")
print("="*80)
sensitivity_general = analyze_sensitivity_to_proportion(df)
for scen in escenarios:
    analyze_sensitivity_to_proportion(df, scen)
print()

# ====================================================================================
# 9. RESUMEN EJECUTIVO CON BONFERRONI
# ====================================================================================

print("="*80)
print("RESUMEN EJECUTIVO")
print("="*80)

print("\nProporción óptima más frecuente:")
print("-" * 60)
optimal_counts = optimal_general['Proporcion_Optima_%'].value_counts()
for prop, count in optimal_counts.items():
    print(f"  {prop}%: {count} modelos")

print(f"\nTop 5 modelos con menor sensibilidad (más robustos):")
print("-" * 60)
top_5_robust = sensitivity_general.nsmallest(5, 'Rango_Relativo_%')
for _, row in top_5_robust.iterrows():
    print(f"  {row['Modelo']:30s}: {row['Rango_Relativo_%']:6.2f}% variación")

print(f"\nTop 5 modelos con mayor sensibilidad:")
print("-" * 60)
top_5_sensitive = sensitivity_general.nlargest(5, 'Rango_Relativo_%')
for _, row in top_5_sensitive.iterrows():
    print(f"  {row['Modelo']:30s}: {row['Rango_Relativo_%']:6.2f}% variación")

# Análisis de significancia con Bonferroni
print(f"\nAnálisis de Significancia Estadística (Bonferroni):")
print("-" * 60)

# Calcular proporciones extremas
prop_min = min(prop_values)
prop_max = max(prop_values)

# Filtrar comparaciones extremas
comparaciones_extremas = dm_results_general[
    (dm_results_general['Proporcion_1_%'] == prop_min) & 
    (dm_results_general['Proporcion_2_%'] == prop_max)
]

if len(comparaciones_extremas) > 0:
    # Calcular alpha bonferroni
    n_comparisons = len(dm_results_general)
    alpha_bonf = 0.05 / n_comparisons
    
    print(f"\nComparación {prop_min}% vs {prop_max}%:")
    print(f"  α corregido (Bonferroni): {alpha_bonf:.8f}")
    
    # Significativas con Bonferroni
    sig_bonf = comparaciones_extremas[
        comparaciones_extremas['Significativo_Bonferroni'].str.startswith('Sí')
    ].sort_values('Mejora_%', ascending=False)
    
    print(f"\n  Modelos con diferencias significativas (post-Bonferroni): {len(sig_bonf)}/{len(comparaciones_extremas)}")
    
    if len(sig_bonf) > 0:
        print(f"\n  Top 5 mayores mejoras significativas (Bonferroni):")
        for _, row in sig_bonf.head(5).iterrows():
            print(f"    {row['Modelo']:30s}: {row['Mejora_%']:6.2f}% - {row['Significativo_Bonferroni']}")
    else:
        print("    No hay diferencias significativas tras corrección de Bonferroni")
    
    # Comparación sin corrección vs con corrección
    sig_sin_corr = comparaciones_extremas[
        comparaciones_extremas['p_value'] < 0.05
    ]
    
    print(f"\n  Comparación de tasas de significancia:")
    print(f"    Sin corrección (p<0.05): {len(sig_sin_corr)}/{len(comparaciones_extremas)} ({len(sig_sin_corr)/len(comparaciones_extremas)*100:.1f}%)")
    print(f"    Con Bonferroni: {len(sig_bonf)}/{len(comparaciones_extremas)} ({len(sig_bonf)/len(comparaciones_extremas)*100:.1f}%)")
    
    if len(sig_sin_corr) > 0:
        tasa_supervivencia = len(sig_bonf) / len(sig_sin_corr) * 100
        print(f"    Tasa de supervivencia: {tasa_supervivencia:.1f}%")
    
    # Mostrar todos los resultados con ambas clasificaciones
    print(f"\n  Resumen completo de comparaciones {prop_min}% vs {prop_max}%:")
    print(f"  {'Modelo':<30} {'Mejora%':>8} {'p-value':>12} {'Sin Corr.':<18} {'Bonferroni':<20}")
    print("  " + "-" * 90)
    
    for _, row in comparaciones_extremas.sort_values('Mejora_%', ascending=False).iterrows():
        print(f"  {row['Modelo']:<30} {row['Mejora_%']:>8.2f} {row['p_value']:>12.6f} "
              f"{row['Significativo_Sin_Corrección']:<18} {row['Significativo_Bonferroni']:<20}")

print("\n" + "="*80)
print("ANÁLISIS COMPLETADO EXITOSAMENTE")
print("="*80)
print(f"Resultados guardados en: {output_dir}")
print(f"\nArchivos generados:")
print(f"  - 4 Heatmaps con Z-scores (1 general + 3 por escenario)")
print(f"  - 4 Archivos Excel con Test DM y Bonferroni (1 general + 3 por escenario)")
print(f"    * Cada Excel incluye hoja de resumen con estadísticas de Bonferroni")
print(f"  - 4 Gráficas de evolución (1 general + 3 por escenario)")
print(f"  - 4 Análisis de proporción óptima (1 general + 3 por escenario)")
print(f"  - 4 Análisis de sensibilidad (1 general + 3 por escenario)")
print("="*80 + "\n")


SIMULACIÓN 4: PROPORCIONES DE CALIBRACIÓN (N=240 FIJO)
CON TEST DIEBOLD-MARIANO MODIFICADO Y CORRECCIÓN DE BONFERRONI

Modelos evaluados: 9
Proporciones: [np.int64(10), np.int64(20), np.int64(30), np.int64(40), np.int64(50)]
Escenarios: ['Lineal Estacionario (ARMA)', 'Lineal No Estacionario (ARIMA)', 'No Lineal Estacionario (SETAR)']

1. GENERANDO HEATMAPS: Z-scores por Modelo
✓ Heatmap Z-scores generado: General
✓ Heatmap Z-scores generado: Lineal_Estacionario_ARMA
✓ Heatmap Z-scores generado: Lineal_No_Estacionario_ARIMA
✓ Heatmap Z-scores generado: No_Lineal_Estacionario_SETAR

2. TEST DM CON CORRECCIÓN DE BONFERRONI

CORRECCIÓN DE BONFERRONI - General
Número de modelos: 9
Comparaciones por modelo: 10
Total de comparaciones: 90
α nominal: 0.05
α corregido (Bonferroni): 0.00055556

✓ Test DM con Bonferroni (Excel) generado: General
  - 90 comparaciones totales
  - 2 significativas sin corrección (p<0.05)
  - 0 significativas con Bonferroni
  - 0.00% tasa de supervivencia


CORRECCIÓ

# Analisis Simulacion h

### Preprocesamiento

In [ ]:
import pandas as pd

# 1. Leer los archivos
arma_df = pd.read_excel("./datos/Simulacion_h/resultados_140_trayectorias_ARMA_FINAL.xlsx")
arima_df = pd.read_excel("./datos/Simulacion_h/resultados_140_trayectorias_ARIMA_FINAL.xlsx")
setar_df = pd.read_excel("./datos/Simulacion_h/resultados_140_trayectorias_SETAR_FINAL.xlsx")
LSPMW_df = pd.read_excel("./datos/Simulacion_h/resultados_LSPMW_todos_procesos.xlsx")


# 2. Asignar la columna ESCENARIO a cada dataframe
arma_df['ESCENARIO'] = "Lineal Estacionario"
arima_df['ESCENARIO'] = "Lineal No estacionario"
setar_df['ESCENARIO'] = "No lineal Estacionario"

# 3. Juntarlos uno bajo el otro (Concatenar)
df_total = pd.concat([arma_df, arima_df, setar_df], ignore_index=True)

# Seleccionar solo las columnas requeridas
columnas_deseadas = [
    "Paso", "Config", "Dist", "Var",  
    "Sieve Bootstrap", "LSPM",  "MCPS", 
    "DeepAR", "ESCENARIO"
]
df_total = df_total[columnas_deseadas]

# Definimos cuáles son las columnas que representan a los modelos predictivos
modelos = [
    "Sieve Bootstrap", "LSPM",  "MCPS", 
    "DeepAR"
]

# 4. Guardar el dataframe consolidado
df_total.to_excel("./datos/Simulacion_h/dataframe_consolidado.xlsx", index=False)

# 5. Generar y mostrar las tablas (Media y Mediana)
metricas = {'MEDIA': 'mean', 'MEDIANA': 'median'}

for nombre_metrica, funcion in metricas.items():
    # Calculamos el valor general según la métrica (mean o median)
    if funcion == 'mean':
        resumen_general = df_total[modelos].mean()
        resumen_escenarios = df_total.groupby('ESCENARIO')[modelos].mean().T
    else:
        resumen_general = df_total[modelos].median()
        resumen_escenarios = df_total.groupby('ESCENARIO')[modelos].median().T

    # Construimos la tabla final para esta métrica
    tabla_resumen = pd.DataFrame(index=modelos)
    tabla_resumen['General'] = resumen_general
    tabla_resumen['ARMA'] = resumen_escenarios['Lineal Estacionario']
    tabla_resumen['ARIMA'] = resumen_escenarios['Lineal No estacionario']
    tabla_resumen['SETAR'] = resumen_escenarios['No lineal Estacionario']

    # Determinar el Mejor_Escenario (valor mínimo entre los tres escenarios)
    escenarios_cols = ['ARMA', 'ARIMA', 'SETAR']
    tabla_resumen['Mejor_Escenario'] = tabla_resumen[escenarios_cols].idxmin(axis=1)

    # Imprimir resultado
    print(f"\n--- Tabla Comparativa de Modelos (Basada en {nombre_metrica}) ---")
    print(tabla_resumen.reset_index().rename(columns={'index': 'Modelo'}).to_string(index=False))


--- Tabla Comparativa de Modelos (Basada en MEDIA) ---
         Modelo  General     ARMA    ARIMA    SETAR Mejor_Escenario
Sieve Bootstrap 1.261013 0.793567 2.405825 0.583647           SETAR
           LSPM 1.423114 0.863196 2.811590 0.594555           SETAR
           MCPS 2.104899 0.881556 4.823525 0.609616           SETAR
         DeepAR 3.126949 0.833898 7.970984 0.575966           SETAR

--- Tabla Comparativa de Modelos (Basada en MEDIANA) ---
         Modelo  General     ARMA    ARIMA    SETAR Mejor_Escenario
Sieve Bootstrap 0.722924 0.622694 1.688908 0.497262           SETAR
           LSPM 0.786162 0.661725 1.946193 0.502726           SETAR
           MCPS 0.847082 0.659495 2.883693 0.522937           SETAR
         DeepAR 0.821647 0.635593 3.456565 0.473015           SETAR


### Analisis

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import normaltest
import warnings
from pathlib import Path
import itertools

warnings.filterwarnings('ignore')

# Configuración general
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'
sns.set_palette("husl")

# Crear carpeta de resultados
output_dir = Path("./Resultados_analisis/Simulacion_h")
output_dir.mkdir(parents=True, exist_ok=True)

# Crear carpeta de interacciones
interactions_dir = output_dir / "Interacciones"
interactions_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_excel("./datos/Simulacion_h/dataframe_consolidado.xlsx")

# 1) CAMBIO DE NOMBRES DE ESCENARIOS
df['ESCENARIO'] = df['ESCENARIO'].replace({
    "Lineal Estacionario": "Lineal Estacionario (ARMA)",
    "Lineal No estacionario": "Lineal No Estacionario (ARIMA)",
    "No lineal Estacionario": "No lineal Estacionario (SETAR)"
})

# Identificar columnas de modelos
var_cols = ['Paso', 'Config', 'Dist', 'Var', 'ESCENARIO']
original_model_cols = [col for col in df.columns if col not in var_cols]

# 2) ORGANIZAR MODELOS POR RENDIMIENTO EN "Lineal No Estacionario (ARIMA)" (Menor a mayor)
target_scenario = "Lineal No Estacionario (ARIMA)"
model_order_scores = df[df['ESCENARIO'] == target_scenario][original_model_cols].mean().sort_values()
model_cols = list(model_order_scores.index)

print("Nuevo orden de modelos (basado en Lineal No Estacionario (ARIMA)):")
for i, m in enumerate(model_cols, 1):
    print(f"{i}. {m}: {model_order_scores[m]:.4f}")

# Mapeo de escenarios
escenarios_map = {
    'Lineal Estacionario (ARMA)': 'Lineal Estacionario (ARMA)',
    'Lineal No Estacionario (ARIMA)': 'Lineal No Estacionario (ARIMA)',
    'No lineal Estacionario (SETAR)': 'No lineal Estacionario (SETAR)'
}

# Definir colores para cada modelo
palette = sns.color_palette("husl", len(model_cols))
model_colors = {model: palette[i] for i, model in enumerate(model_cols)}

# ====================================================================================
# SECCIÓN 1: RENDIMIENTO POR ESCENARIOS
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 1: RENDIMIENTO POR ESCENARIOS")
print("="*80)

def plot_performance_by_scenario():
    fig, ax = plt.subplots(figsize=(14, 8))
    
    scenarios = ["Lineal Estacionario (ARMA)", "Lineal No Estacionario (ARIMA)", "No lineal Estacionario (SETAR)"]
    x = np.arange(len(model_cols))
    width = 0.25 
    
    scenario_colors = {
        'Lineal Estacionario (ARMA)': '#5D3FD3',    
        'Lineal No Estacionario (ARIMA)': '#808080', 
        'No lineal Estacionario (SETAR)': '#00A36C'  
    }
    
    for idx, scenario in enumerate(scenarios):
        means = [df[df['ESCENARIO'] == scenario][model].mean() for model in model_cols]
        position = x + (idx - 1) * width
        
        bars = ax.bar(position, means, width, label=scenario, 
                     color=scenario_colors[scenario], alpha=0.85, edgecolor='black', linewidth=0.5)
        
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2f}', ha='center', va='bottom', fontsize=7, rotation=0)
    
    ax.set_xlabel('Modelo (Ordenados por desempeño en ARIMA)', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title('Rendimiento de Modelos por Escenario (ECRPS)', fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(model_cols, rotation=45, ha='right', fontsize=9)
    ax.legend(loc='upper left', ncol=1, fontsize=10, framealpha=0.9)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '1.1_rendimiento_por_escenario.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 1.1 guardada")

plot_performance_by_scenario()

def plot_relative_performance():
    base_scenario = 'Lineal Estacionario (ARMA)'
    scenarios_compare = ['Lineal No Estacionario (ARIMA)', 'No lineal Estacionario (SETAR)']
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    y = np.arange(len(model_cols))
    height = 0.35  
    
    for idx, scenario in enumerate(scenarios_compare):
        changes = []
        for model in model_cols:
            base_value = df[df['ESCENARIO'] == base_scenario][model].mean()
            scenario_value = df[df['ESCENARIO'] == scenario][model].mean()
            pct_change = ((scenario_value - base_value) / base_value) * 100
            changes.append(pct_change)
        
        position = y + idx * height
        bars = ax.barh(position, changes, height, label=scenario, alpha=0.85, edgecolor='black', linewidth=0.5)
        
        for bar, val in zip(bars, changes):
            width = bar.get_width()
            ax.text(width + (1 if width > 0 else -1), bar.get_y() + bar.get_height()/2.,
                   f'{val:+.1f}%', ha='left' if val > 0 else 'right', 
                   va='center', fontsize=7)
    
    ax.set_yticks(y + height / 2)
    ax.set_yticklabels(model_cols, fontsize=10)
    ax.set_xlabel('Cambio Relativo (%)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title(f'Cambio Relativo en ECRPS vs. {base_scenario}', fontsize=14, fontweight='bold', pad=20)
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1.5)
    ax.legend(loc='best', fontsize=10, framealpha=0.9)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '1.2_cambio_relativo_escenario_base.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 1.2 guardada")

plot_relative_performance()

# ====================================================================================
# SECCIÓN 2: ANÁLISIS POR CONFIG
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 2: ANÁLISIS POR CONFIG")
print("="*80)

def plot_zscore_heatmap_config(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Z-scores de ECRPS por Configuración ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Z-scores de ECRPS por Configuración (General)'
    
    pivot_data = data_filtered.groupby('Config')[model_cols].mean()
    z_scores = pivot_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(z_scores.T, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, cbar_kws={'label': 'Z-score'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=12, fontweight='bold', pad=20)
    ax.set_xlabel('Configuración', fontsize=11)
    ax.set_ylabel('Modelo', fontsize=11)
    
    plt.tight_layout()
    filename = f'2.1{suffix}_zscore_config.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 2.1{suffix} guardada")

plot_zscore_heatmap_config()
plot_zscore_heatmap_config('Lineal Estacionario (ARMA)', '.a')
plot_zscore_heatmap_config('Lineal No Estacionario (ARIMA)', '.b')
plot_zscore_heatmap_config('No lineal Estacionario (SETAR)', '.c')

def plot_variability_config(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Configuración ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Configuración (General)'
    
    pivot_std = data_filtered.groupby('Config')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    fmt = '.2f' if suffix == '' else '.4f'
    sns.heatmap(pivot_std.T, annot=True, fmt=fmt, cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Configuración', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'2.2{suffix}_variabilidad_config.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 2.2{suffix} guardada")

plot_variability_config()
plot_variability_config('Lineal Estacionario (ARMA)', '.a')
plot_variability_config('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_config('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 3: ANÁLISIS POR DIST
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 3: ANÁLISIS POR DIST")
print("="*80)

def plot_zscore_heatmap_dist(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Z-scores de ECRPS por Distribución ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Z-scores de ECRPS por Distribución (General)'
    
    pivot_data = data_filtered.groupby('Dist')[model_cols].mean()
    z_scores = pivot_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(z_scores.T, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, cbar_kws={'label': 'Z-score'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=12, fontweight='bold', pad=20)
    ax.set_xlabel('Distribución', fontsize=11)
    ax.set_ylabel('Modelo', fontsize=11)
    
    plt.tight_layout()
    filename = f'3.1{suffix}_zscore_dist.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 3.1{suffix} guardada")

plot_zscore_heatmap_dist()
plot_zscore_heatmap_dist('Lineal Estacionario (ARMA)', '.a')
plot_zscore_heatmap_dist('Lineal No Estacionario (ARIMA)', '.b')
plot_zscore_heatmap_dist('No lineal Estacionario (SETAR)', '.c')

def plot_variability_dist(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Distribución ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Distribución (General)'
    
    pivot_std = data_filtered.groupby('Dist')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Distribución', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'3.2{suffix}_variabilidad_dist.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 3.2{suffix} guardada")

plot_variability_dist()
plot_variability_dist('Lineal Estacionario (ARMA)', '.a')
plot_variability_dist('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_dist('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 4: ANÁLISIS POR VAR
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 4: ANÁLISIS POR VAR")
print("="*80)

def plot_evolution_var(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Evolución de ECRPS por Varianza ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Evolución de ECRPS por Varianza (General)'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    var_values = sorted(data_filtered['Var'].unique())
    
    for model in model_cols:
        means = []
        for var in var_values:
            mean_val = data_filtered[data_filtered['Var'] == var][model].mean()
            means.append(mean_val)
        
        ax.plot(var_values, means, marker='o', label=model, color=model_colors[model],
                linewidth=2.5, markersize=7, alpha=0.85)
    
    ax.set_xlabel('Varianza', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=9, ncol=2, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    filename = f'4.1{suffix}_evolucion_var.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 4.1{suffix} guardada")

plot_evolution_var()
plot_evolution_var('Lineal Estacionario (ARMA)', '.a')
plot_evolution_var('Lineal No Estacionario (ARIMA)', '.b')
plot_evolution_var('No lineal Estacionario (SETAR)', '.c')

def plot_variability_var(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Varianza ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Varianza (General)'
    
    pivot_std = data_filtered.groupby('Var')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Varianza', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'4.2{suffix}_variabilidad_var.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 4.2{suffix} guardada")

plot_variability_var()
plot_variability_var('Lineal Estacionario (ARMA)', '.a')
plot_variability_var('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_var('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)")
print("="*80)

def plot_evolution_paso(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Evolución de ECRPS por Horizonte de Pronóstico ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Evolución de ECRPS por Horizonte de Pronóstico (General)'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    pasos = sorted(data_filtered['Paso'].unique())
    
    for model in model_cols:
        means = []
        for paso in pasos:
            mean_val = data_filtered[data_filtered['Paso'] == paso][model].mean()
            means.append(mean_val)
        
        ax.plot(pasos, means, marker='o', label=model, color=model_colors[model],
                linewidth=2.5, markersize=7, alpha=0.85)
    
    ax.set_xlabel('Horizonte de Pronóstico', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=9, ncol=2, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    filename = f'5.1{suffix}_evolucion_paso.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 5.1{suffix} guardada")

plot_evolution_paso()
plot_evolution_paso('Lineal Estacionario (ARMA)', '.a')
plot_evolution_paso('Lineal No Estacionario (ARIMA)', '.b')
plot_evolution_paso('No lineal Estacionario (SETAR)', '.c')

def plot_variability_paso(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Horizonte ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Horizonte (General)'
    
    pivot_std = data_filtered.groupby('Paso')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Horizonte de Pronóstico', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'5.2{suffix}_variabilidad_paso.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 5.2{suffix} guardada")

plot_variability_paso()
plot_variability_paso('Lineal Estacionario (ARMA)', '.a')
plot_variability_paso('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_paso('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 5.5: ANÁLISIS DE INTERACCIONES
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5.5: ANÁLISIS DE INTERACCIONES")
print("="*80)

# Función auxiliar para configurar el grid de modelos
def get_model_grid_axes(n_models):
    n_cols = 3
    n_rows = (n_models + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
    return fig, axes.flatten(), n_rows, n_cols

# INTERACCIÓN 3 (Lista 8): DIST × PASO - Subplots por Modelo
def plot_interaction_dist_paso(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Dist × Paso por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, n_rows, n_cols = get_model_grid_axes(len(model_cols))
    dists = sorted(data_filtered['Dist'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_dist = sns.color_palette("viridis", len(dists))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for d_idx, dist in enumerate(dists):
            means = [data_filtered[(data_filtered['Dist'] == dist) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='o', label=dist, color=colors_dist[d_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Distribución", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.7{suffix}_interaccion_dist_paso_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 8 (5.7) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 4 (Lista 9): DIST × VAR - Subplots por Modelo
def plot_interaction_dist_var(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Dist × Var por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    dists = sorted(data_filtered['Dist'].unique())
    vars_val = sorted(data_filtered['Var'].unique())
    colors_dist = sns.color_palette("magma", len(dists))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for d_idx, dist in enumerate(dists):
            means = [data_filtered[(data_filtered['Dist'] == dist) & (data_filtered['Var'] == v)][model].mean() for v in vars_val]
            ax.plot(vars_val, means, marker='s', label=dist, color=colors_dist[d_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Varianza')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Distribución", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.8{suffix}_interaccion_dist_var_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 9 (5.8) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 5 (Lista 10): CONFIG × PASO - Subplots por Modelo
def plot_interaction_config_paso(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Config × Paso por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    configs = sorted(data_filtered['Config'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_conf = sns.color_palette("tab10", len(configs))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for c_idx, config in enumerate(configs):
            means = [data_filtered[(data_filtered['Config'] == config) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='^', label=config, color=colors_conf[c_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Config", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.9{suffix}_interaccion_config_paso_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 10 (5.9) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 6 (Lista 11): VAR × HORIZONTE - Subplots por Modelo
def plot_interaction_var_horizonte(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Var × Horizonte por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    vars_val = sorted(data_filtered['Var'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_var = sns.color_palette("rocket", len(vars_val))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for v_idx, var in enumerate(vars_val):
            means = [data_filtered[(data_filtered['Var'] == var) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='d', label=f'Var {var}', color=colors_var[v_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Varianza", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.10{suffix}_interaccion_var_horizonte_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 11 (5.10) {suffix} guardada: Subplots por Modelo")


# ====================================================================================
# EJECUCIÓN DE LAS NUEVAS FUNCIONES
# ====================================================================================

for sc_name, sc_suf in [ (None, ''), ('Lineal Estacionario (ARMA)', '.a'), 
                        ('Lineal No Estacionario (ARIMA)', '.b'), 
                        ('No lineal Estacionario (SETAR)', '.c') ]:
    plot_interaction_dist_paso(sc_name, sc_suf)
    plot_interaction_dist_var(sc_name, sc_suf)
    plot_interaction_config_paso(sc_name, sc_suf)
    plot_interaction_var_horizonte(sc_name, sc_suf)

# ====================================================================================
# SECCIÓN 6: ROBUSTEZ Y TEST DIEBOLD-MARIANO
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 6: ROBUSTEZ Y TEST DIEBOLD-MARIANO")
print("="*80)

def plot_robustness():
    fig, ax = plt.subplots(figsize=(12, 8))
    
    cv_data = []
    for model in model_cols:
        cv = df[model].std() / df[model].mean()
        cv_data.append((model, cv))
    
    cv_df = pd.DataFrame(cv_data, columns=['Modelo', 'CV'])
    
    # CAMBIO 1: Ordenar de menor a mayor CV para coherencia
    cv_df = cv_df.sort_values('CV')
    
    colors_cv = ['#2ecc71' if cv < cv_df['CV'].median() else '#e74c3c' 
                 for cv in cv_df['CV']]
    
    bars = ax.barh(cv_df['Modelo'], cv_df['CV'], color=colors_cv, alpha=0.8, edgecolor='black')
    
    for bar, cv in zip(bars, cv_df['CV']):
        width = bar.get_width()
        ax.text(width + 0.001, bar.get_y() + bar.get_height()/2.,
               f'{cv:.4f}', ha='left', va='center', fontsize=9)
    
    ax.set_xlabel('Coeficiente de Variación', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title('Robustez: Coeficiente de Variación\n(Ordenado de menor a mayor - Menor valor indica mayor estabilidad)', 
                  fontsize=14, fontweight='bold', pad=20)
    ax.axvline(x=cv_df['CV'].median(), color='black', linestyle='--', linewidth=1.5, alpha=0.5, label='Mediana')
    ax.legend(loc='best', fontsize=10)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '6.1_robustez_coeficiente_variacion.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 6.1 guardada (ordenada de menor a mayor CV)")

plot_robustness()

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano con fixed-smoothing asymptotics (Coroneo & Iacone, 2020)
    
    Usa Fixed-m asymptotics (kernel Daniell) que es más robusto en muestras pequeñas
    y mantiene buen desempeño en muestras grandes.
    
    Parameters:
    -----------
    errors1, errors2 : array-like
        Errores de pronóstico (ECRPS) de los dos modelos
    h : int
        Horizonte de pronóstico (forecast horizon)
    
    Returns:
    --------
    hln_dm_stat : float
        Estadístico con fixed-m asymptotics
    p_value : float
        P-valor usando distribución t-Student con 2m grados de libertad
    dm_stat : float
        Estadístico DM original (para referencia)
    """
    # Calcular diferencial de pérdida
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    if T < 2:
        return np.nan, np.nan, np.nan
    
    # Desviaciones de la media
    u = d - d_bar
    
    # Fixed-m: bandwidth recomendado en el paper
    m = max(1, int(np.floor(T**(1/3))))
    
    # Calcular periodograma
    from scipy.fft import fft
    
    # FFT de las desviaciones
    fft_u = fft(u)
    periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
    
    # WPE con kernel de Daniell: promedio de primeros m periodogramas
    # (excluyendo frecuencia 0)
    if m >= len(periodogram) - 1:
        m = len(periodogram) - 2
    
    sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
    
    if sigma_hat_sq <= 0:
        # Fallback: usar varianza simple
        sigma_hat_sq = np.var(d, ddof=1) / T
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
    
    # Estadístico DM
    dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
    
    # Fixed-m asymptotics: límite es t-Student con 2m grados de libertad
    df = 2 * m
    hln_dm_stat = dm_stat
    
    # P-valor usando t-Student
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat

def plot_dm_test_heatmap(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Test Diebold-Mariano Modificado (HLN-DM)\ncon Corrección de Bonferroni ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Test Diebold-Mariano Modificado (HLN-DM)\ncon Corrección de Bonferroni (General)'
    
    # Determinar el horizonte de pronóstico promedio
    h_forecast = int(data_filtered['Paso'].mean())
    
    n_models = len(model_cols)
    n_comparisons = n_models * (n_models - 1) / 2
    alpha = 0.05
    bonferroni_alpha = alpha / n_comparisons
    
    results_matrix = np.zeros((n_models, n_models))
    p_values = np.zeros((n_models, n_models))
    dm_stats = np.zeros((n_models, n_models))
    
    for i, model1 in enumerate(model_cols):
        for j, model2 in enumerate(model_cols):
            if i == j:
                results_matrix[i, j] = 0  
                p_values[i, j] = 1.0
                dm_stats[i, j] = 0
            elif i < j:
                errors1 = data_filtered[model1].values
                errors2 = data_filtered[model2].values
                hln_dm_stat, p_val, dm_original = modified_diebold_mariano_test(
                    errors1, errors2, h=h_forecast
                )
                
                p_values[i, j] = p_val
                p_values[j, i] = p_val
                dm_stats[i, j] = hln_dm_stat
                dm_stats[j, i] = -hln_dm_stat
                
                if p_val < bonferroni_alpha:
                    mean1 = np.mean(errors1)
                    mean2 = np.mean(errors2)
                    if mean1 < mean2:  
                        results_matrix[i, j] = 1  
                        results_matrix[j, i] = -1  
                    else:
                        results_matrix[i, j] = -1
                        results_matrix[j, i] = 1
                else:
                    results_matrix[i, j] = 0
                    results_matrix[j, i] = 0
    
    fig, ax = plt.subplots(figsize=(14, 11))
    cmap = plt.cm.colors.ListedColormap(['#e74c3c', '#fff9c4', '#2ecc71'])
    bounds = [-1.5, -0.5, 0.5, 1.5]
    norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)
    
    im = ax.imshow(results_matrix, cmap=cmap, norm=norm, aspect='auto')
    ax.set_xticks(np.arange(n_models))
    ax.set_yticks(np.arange(n_models))
    ax.set_xticklabels(model_cols, rotation=45, ha='right', fontsize=9)
    ax.set_yticklabels(model_cols, fontsize=9)
    
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                text = '-'
                color = 'black'
            else:
                val = results_matrix[i, j]
                p_val = p_values[i, j]
                dm_val = dm_stats[i, j]
                if val == 1:
                    text = f'✓\nHLN-DM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'white'
                elif val == -1:
                    text = f'✗\nHLN-DM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'white'
                else:
                    text = f'≈\nHLN-DM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'black'
            ax.text(j, i, text, ha='center', va='center', 
                   color=color, fontsize=6, fontweight='bold')
    
    # Obtener T para el título
    T = len(data_filtered)
    df_test = T - 1
    
    ax.set_title(title + f'\n(h={h_forecast}, T={T}, df={df_test}, α ajustado={bonferroni_alpha:.5f})', 
                 fontsize=11, fontweight='bold', pad=20)
    ax.set_xlabel('Modelo (Columna)', fontsize=11)
    ax.set_ylabel('Modelo (Fila)', fontsize=11)
    
    legend_elements = [
        plt.Rectangle((0,0),1,1, facecolor='#2ecc71', label='Fila supera columna (p < α)'),
        plt.Rectangle((0,0),1,1, facecolor='#e74c3c', label='Columna supera fila (p < α)'),
        plt.Rectangle((0,0),1,1, facecolor='#fff9c4', label='Sin diferencia significativa (p ≥ α)')
    ]
    ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=9)
    
    # Agregar nota metodológica
    note_text = ('Nota: Se utiliza el test HLN-DM con corrección para muestras finitas.\n'
                 'Distribución: t-Student. Ajuste: Bonferroni para comparaciones múltiples.')
    plt.figtext(0.5, -0.02, note_text, ha='center', fontsize=8, style='italic', 
                wrap=True, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.tight_layout()
    filename = f'6.2{suffix}_dm_test_hln_bonferroni.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    
    # Guardar resultados en Excel
    results_df = pd.DataFrame(results_matrix, index=model_cols, columns=model_cols)
    pvalues_df = pd.DataFrame(p_values, index=model_cols, columns=model_cols)
    dmstats_df = pd.DataFrame(dm_stats, index=model_cols, columns=model_cols)
    
    summary_data = []
    for model in model_cols:
        idx = model_cols.index(model)
        victorias = int(np.sum(results_matrix[idx, :] == 1))
        derrotas = int(np.sum(results_matrix[idx, :] == -1))
        empates = int(np.sum(results_matrix[idx, :] == 0)) - 1
        mean_ecrps = data_filtered[model].mean()
        std_ecrps = data_filtered[model].std()
        cv_ecrps = std_ecrps / mean_ecrps
        summary_data.append({
            'Modelo': model,
            'Victorias': victorias,
            'Derrotas': derrotas,
            'Empates': empates,
            'Tasa_Victoria': f"{(victorias / (n_models - 1)) * 100:.1f}%",
            'ECRPS_Promedio': f"{mean_ecrps:.4f}",
            'ECRPS_Desv_Std': f"{std_ecrps:.4f}",
            'Coef_Variacion': f"{cv_ecrps:.4f}"
        })
    summary_df = pd.DataFrame(summary_data)
    
    # Información metodológica
    method_info = pd.DataFrame({
        'Parámetro': ['Horizonte de pronóstico (h)', 'Tamaño muestra (T)', 
                      'Grados libertad (df)', 'Alpha nominal', 
                      'Alpha ajustado (Bonferroni)', 'Número comparaciones',
                      'Distribución', 'Corrección aplicada'],
        'Valor': [h_forecast, T, df_test, alpha, bonferroni_alpha, 
                  int(n_comparisons), 't-Student', 'Harvey-Leybourne-Newbold (1997)']
    })
    
    with pd.ExcelWriter(output_dir / f'6.2{suffix}_dm_test_hln_resultados.xlsx') as writer:
        method_info.to_excel(writer, sheet_name='Metodologia', index=False)
        results_df.to_excel(writer, sheet_name='Matriz_Resultados')
        pvalues_df.to_excel(writer, sheet_name='P_valores')
        dmstats_df.to_excel(writer, sheet_name='Estadisticos_HLN_DM')
        summary_df.to_excel(writer, sheet_name='Resumen_Modelos', index=False)
    
    print(f"✓ Gráfica 6.2{suffix} guardada (Test HLN-DM con h={h_forecast}, T={T}, df={df_test})")

plot_dm_test_heatmap()
plot_dm_test_heatmap('Lineal Estacionario (ARMA)', '.a')
plot_dm_test_heatmap('Lineal No Estacionario (ARIMA)', '.b')
plot_dm_test_heatmap('No lineal Estacionario (SETAR)', '.c')

print("\n" + "="*80)
print("PROCESO COMPLETADO")
print("="*80)
print("\nMejoras implementadas:")
print("1. ✓ Orden consistente en gráficas 1.1 y 1.2")
print("2. ✓ Formato de 2 decimales en heatmap 2.2 general")
print("3. ✓ Solo Coeficiente de Variación en gráfica 6.1 (ordenado menor a mayor)")
print("4. ✓ Test Diebold-Mariano con corrección HLN y distribución t-Student")
print("5. ✓ Horizonte de pronóstico (h) incorporado en el análisis")
print("\n" + "="*80)
print("ANÁLISIS DE INTERACCIONES (Carpeta: Interacciones/)")
print("="*80)
print("6. ✓ Config × Var: Heatmaps por modelo (matriz 3×3)")
print("7. ✓ Config × Dist: Heatmaps por modelo (matriz 3×3)")
print("8. ✓ Dist × Paso: Gráficas de líneas por distribución")
print("9. ✓ Dist × Var: Gráficas de líneas por distribución")
print("10. ✓ Config × Paso: Gráficas de líneas por configuración")
print("11. ✓ Var × Horizonte: Gráficas de líneas por varianza")
print("="*80)
print(f"\nTotal de gráficas generadas en Interacciones/: {6 * 4} archivos")
print("(6 tipos de interacción × 4 escenarios: General + 3 específicos)")
print("="*80)

Nuevo orden de modelos (basado en Lineal No Estacionario (ARIMA)):
1. Sieve Bootstrap: 2.4058
2. LSPM: 2.8116
3. MCPS: 4.8235
4. DeepAR: 7.9710

SECCIÓN 1: RENDIMIENTO POR ESCENARIOS
✓ Gráfica 1.1 guardada
✓ Gráfica 1.2 guardada

SECCIÓN 2: ANÁLISIS POR CONFIG
✓ Gráfica 2.1 guardada
✓ Gráfica 2.1.a guardada
✓ Gráfica 2.1.b guardada
✓ Gráfica 2.1.c guardada
✓ Gráfica 2.2 guardada
✓ Gráfica 2.2.a guardada
✓ Gráfica 2.2.b guardada
✓ Gráfica 2.2.c guardada

SECCIÓN 3: ANÁLISIS POR DIST
✓ Gráfica 3.1 guardada
✓ Gráfica 3.1.a guardada
✓ Gráfica 3.1.b guardada
✓ Gráfica 3.1.c guardada
✓ Gráfica 3.2 guardada
✓ Gráfica 3.2.a guardada
✓ Gráfica 3.2.b guardada
✓ Gráfica 3.2.c guardada

SECCIÓN 4: ANÁLISIS POR VAR
✓ Gráfica 4.1 guardada
✓ Gráfica 4.1.a guardada
✓ Gráfica 4.1.b guardada
✓ Gráfica 4.1.c guardada
✓ Gráfica 4.2 guardada
✓ Gráfica 4.2.a guardada
✓ Gráfica 4.2.b guardada
✓ Gráfica 4.2.c guardada

SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)
✓ Gráfica 5.1 guardada
✓ Gráfica 5.1.a guardada
✓ 

# Revisión uso DM

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# ============================================================================
# SIMULACIÓN DE DATOS SIMPLIFICADA
# ============================================================================
# Supongamos solo 2 modelos, 2 configs, 2 distribuciones, 2 varianzas, 3 pasos
# Total: 2×2×2×3 = 24 observaciones por modelo

np.random.seed(42)

configs = [1, 2]
dists = ['Normal', 'T-Student']
vars_val = [0.5, 1.0]
pasos = [1, 2, 3]

data = []
for config in configs:
    for dist in dists:
        for var in vars_val:
            for paso in pasos:
                # Modelo A: mejor en general, pero peor con Config=2 + Var=1.0
                if config == 2 and var == 1.0:
                    ecrps_A = np.random.normal(0.8, 0.1)  # Peor
                else:
                    ecrps_A = np.random.normal(0.4, 0.05)  # Mejor
                
                # Modelo B: más estable, rendimiento medio
                ecrps_B = np.random.normal(0.5, 0.05)
                
                data.append({
                    'Config': config,
                    'Dist': dist,
                    'Var': var,
                    'Paso': paso,
                    'Modelo_A': ecrps_A,
                    'Modelo_B': ecrps_B
                })

df = pd.DataFrame(data)
print("="*80)
print("DATOS SIMULADOS (primeras 12 filas)")
print("="*80)
print(df.head(12))
print(f"\nTotal observaciones: {len(df)}")

# ============================================================================
# ENFOQUE ACTUAL: UN SOLO TEST D-M CON TODAS LAS OBSERVACIONES
# ============================================================================
print("\n" + "="*80)
print("ENFOQUE 1: TEST D-M GLOBAL (COMO ESTÁ EN TU CÓDIGO)")
print("="*80)

errors_A = df['Modelo_A'].values
errors_B = df['Modelo_B'].values
diferencias = errors_A - errors_B

print(f"\nDiferencias individuales (primeras 12):")
print(diferencias[:12])
print(f"\nMedia de diferencias: {np.mean(diferencias):.4f}")
print(f"Desv. Std de diferencias: {np.std(diferencias, ddof=1):.4f}")

# Test D-M simple
d_bar = np.mean(diferencias)
T = len(diferencias)
se = np.std(diferencias, ddof=1) / np.sqrt(T)
dm_stat = d_bar / se
p_value_dm = 2 * (1 - stats.norm.cdf(abs(dm_stat)))

print(f"\nEstadístico D-M: {dm_stat:.4f}")
print(f"P-valor: {p_value_dm:.4f}")
print(f"Conclusión: {'Modelo A significativamente diferente de B' if p_value_dm < 0.05 else 'No hay diferencia significativa'}")

# ============================================================================
# ENFOQUE ALTERNATIVO 1: TESTS D-M ESTRATIFICADOS
# ============================================================================
print("\n" + "="*80)
print("ENFOQUE 2: TESTS D-M ESTRATIFICADOS POR CONDICIÓN")
print("="*80)

print("\nTests por Config:")
for config in configs:
    subset = df[df['Config'] == config]
    dif = subset['Modelo_A'].values - subset['Modelo_B'].values
    d_bar = np.mean(dif)
    se = np.std(dif, ddof=1) / np.sqrt(len(dif))
    dm_stat = d_bar / se
    p_val = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
    print(f"  Config {config}: media_dif={d_bar:.4f}, DM={dm_stat:.4f}, p={p_val:.4f}")

print("\nTests por Varianza:")
for var in vars_val:
    subset = df[df['Var'] == var]
    dif = subset['Modelo_A'].values - subset['Modelo_B'].values
    d_bar = np.mean(dif)
    se = np.std(dif, ddof=1) / np.sqrt(len(dif))
    dm_stat = d_bar / se
    p_val = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
    print(f"  Var {var}: media_dif={d_bar:.4f}, DM={dm_stat:.4f}, p={p_val:.4f}")

print("\nTests por Config × Var (combinación crítica):")
for config in configs:
    for var in vars_val:
        subset = df[(df['Config'] == config) & (df['Var'] == var)]
        dif = subset['Modelo_A'].values - subset['Modelo_B'].values
        d_bar = np.mean(dif)
        se = np.std(dif, ddof=1) / np.sqrt(len(dif))
        dm_stat = d_bar / se if se > 0 else 0
        p_val = 2 * (1 - stats.norm.cdf(abs(dm_stat))) if se > 0 else 1.0
        print(f"  Config={config}, Var={var}: media_dif={d_bar:.4f}, DM={dm_stat:.4f}, p={p_val:.4f}")

# ============================================================================
# ENFOQUE ALTERNATIVO 2: ANOVA SOBRE LAS DIFERENCIAS
# ============================================================================
print("\n" + "="*80)
print("ENFOQUE 3: ANOVA MULTIFACTORIAL SOBRE DIFERENCIAS")
print("="*80)

df['Diferencia'] = df['Modelo_A'] - df['Modelo_B']

# ANOVA de efectos principales
from scipy.stats import f_oneway

print("\nEfecto de Config:")
grupos_config = [df[df['Config'] == c]['Diferencia'].values for c in configs]
F_stat, p_val = f_oneway(*grupos_config)
print(f"  F={F_stat:.4f}, p={p_val:.4f}")

print("\nEfecto de Var:")
grupos_var = [df[df['Var'] == v]['Diferencia'].values for v in vars_val]
F_stat, p_val = f_oneway(*grupos_var)
print(f"  F={F_stat:.4f}, p={p_val:.4f}")

print("\nEfecto de Dist:")
grupos_dist = [df[df['Dist'] == d]['Diferencia'].values for d in dists]
F_stat, p_val = f_oneway(*grupos_dist)
print(f"  F={F_stat:.4f}, p={p_val:.4f}")

# Interacción Config × Var
print("\nEfecto de interacción Config × Var:")
grupos_interaccion = []
for config in configs:
    for var in vars_val:
        subset = df[(df['Config'] == config) & (df['Var'] == var)]
        grupos_interaccion.append(subset['Diferencia'].values)
F_stat, p_val = f_oneway(*grupos_interaccion)
print(f"  F={F_stat:.4f}, p={p_val:.4f}")

# ============================================================================
# RESUMEN VISUAL
# ============================================================================
print("\n" + "="*80)
print("RESUMEN: MEDIAS DE DIFERENCIAS POR CONDICIÓN")
print("="*80)

pivot = df.pivot_table(values='Diferencia', 
                       index=['Config', 'Var'], 
                       columns='Dist', 
                       aggfunc='mean')
print("\nTabla: Diferencia promedio (Modelo_A - Modelo_B)")
print(pivot.round(4))
print("\nInterpretación:")
print("  - Valores negativos: Modelo A mejor (menor ECRPS)")
print("  - Valores positivos: Modelo B mejor")
print("  - Nota: Config=2 + Var=1.0 tiene diferencias positivas (A peor que B)")

# ============================================================================
# CONCLUSIÓN COMPARATIVA
# ============================================================================
print("\n" + "="*80)
print("CONCLUSIONES COMPARATIVAS")
print("="*80)

print("\n1. TEST D-M GLOBAL (tu código actual):")
print(f"   → Conclusión: Modelo A {'mejor' if d_bar < 0 else 'peor'} que B en promedio")
print(f"   → Problema: NO detecta que en Config=2+Var=1.0 la situación se invierte")

print("\n2. TESTS D-M ESTRATIFICADOS:")
print("   → Conclusión: La superioridad de A depende de las condiciones")
print("   → Ventaja: Identifica dónde cada modelo es mejor")
print("   → Problema: Múltiples tests, ajuste de Bonferroni muy conservador")

print("\n3. ANOVA SOBRE DIFERENCIAS:")
print("   → Conclusión: Hay efectos significativos de Config, Var, y su interacción")
print("   → Ventaja: Cuantifica qué factores afectan las diferencias")
print("   → Limitación: No hace comparación pairwise directa entre modelos")

print("\n" + "="*80)
print("RECOMENDACIÓN PARA TU ANÁLISIS")
print("="*80)
print("""
Deberías combinar los tres enfoques:

1. Test D-M global → Responde: "¿Quién es mejor en general?"
2. Tests D-M por escenario → Responde: "¿Esa conclusión se mantiene en todos los escenarios?"
3. ANOVA sobre diferencias → Responde: "¿Qué factores explican las diferencias?"

Esto te da una historia completa:
- Modelo A es mejor en promedio (D-M global)
- PERO esa ventaja desaparece cuando Config=2 y Var=1.0 (D-M estratificado)
- Los factores Config y Var tienen efectos significativos (ANOVA)
""")

DATOS SIMULADOS (primeras 12 filas)
    Config       Dist  Var  Paso  Modelo_A  Modelo_B
0        1     Normal  0.5     1  0.424836  0.493087
1        1     Normal  0.5     2  0.432384  0.576151
2        1     Normal  0.5     3  0.388292  0.488293
3        1     Normal  1.0     1  0.478961  0.538372
4        1     Normal  1.0     2  0.376526  0.527128
5        1     Normal  1.0     3  0.376829  0.476714
6        1  T-Student  0.5     1  0.412098  0.404336
7        1  T-Student  0.5     2  0.313754  0.471886
8        1  T-Student  0.5     3  0.349358  0.515712
9        1  T-Student  1.0     1  0.354599  0.429385
10       1  T-Student  1.0     2  0.473282  0.488711
11       1  T-Student  1.0     3  0.403376  0.428763

Total observaciones: 24

ENFOQUE 1: TEST D-M GLOBAL (COMO ESTÁ EN TU CÓDIGO)

Diferencias individuales (primeras 12):
[-0.06825108 -0.14376707 -0.10000082 -0.0594111  -0.15060172 -0.0998844
  0.00776213 -0.15813152 -0.16635392 -0.07478602 -0.01542875 -0.02538618]

Media de 